<a href="https://colab.research.google.com/github/RubicaConsulting/Gallo/blob/main/Proyecto_Gallo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estandarización marcas


*   Apothic
*   Barefoot
*   Carlo Rossi
*   Dark Horse
*   Edna Valley
*   Ghost Pines
*   J Vineyards
*   La Marca
*   Louis M Martini
*   MacMurray
*   Orin Swift
*   Prophecy
*   William Hill



# INICIALIZACIÓN

In [ ]:
#Instalar Selenium

# Primeros pasos con Selenium en Google Colab de la academia https://scraping.link/
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!pip install --upgrade gspread



In [2]:
from google.colab import auth

auth.authenticate_user()

#importar credenciales de autenticación

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
sh = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U')
worksheet0 = sh.get_worksheet(0)
rowsInitial = worksheet0.get_all_values()
rowsInitial = len(rowsInitial)

# ARUBA

Proveedor - **Pepia4home** (https://www.pepia4home.com/)
---


***> Marcas:***
*   *Dark Horse*
*   *La Marca*
*   *Apothic*
*   *Orin Swift*
*   *Louis M  Maritini*
*   *Ghost pines*
*   *MacMurray*
*   *William Hill*
*   *Carlo Rossi*



In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
 
gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


brands=[["Dark Horse","dark+horse","Dark Horse"],["La Marca","lamarca","La Marca"], ["Apothic","apothic","Apothic"],["Orin Swift","orin+swift","Orin Swift"],["Louis M Martini", "louis+martini","Louis M. Martini"],["Ghost Pines", "ghost+pines","Ghost Pines"],["MacMurray","macmurray","MacMurray"],["William Hill","william+hill","William Hill"],["Carlo Rossi","carlo+rossi","Carlo Rossi"]]

for keyword in brands:
  try:
    url = f"https://www.pepia4home.com/search?q={keyword[1]}"
    url_data = requests.get(url)
    content = url_data.text
    soup = BeautifulSoup(content, "lxml")
    productsTags = soup.findAll(True, {"class":["sERPSMg"]})
    productsList = []
    
    for product in productsTags:
      try:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("a", class_="sil_d4M").text
          price = product.find("span", class_="sQq4biu").text.strip().replace("ƒ","").replace(".","").replace(",",".")
          link = product.find("a",class_="sil_d4M", href=True)["href"]
          newProduct=[date,"Aruba","pepia4home",keyword[0],name,float(price),link]
          marca = name.lower().find(keyword[2].lower())
          if marca > -1:
            productsList.append(newProduct)
      except Exception as err:
         print("ERROR "+keyword[0]+":"+name, err)

    for product in productsList:
      worksheet.append_row(product)

    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_pepia4home.csv', index=False)
  except Exception as err:
    print("ERROR "+keyword[0]+":", err)


Proveedor - **Superfood Aruba** (https://shop.superfoodaruba.com/)
---


***> Marcas:***
*   *Barefoot*
* *Carlo Rossi*


In [4]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import numpy as np

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')# max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--incognito')

driver = webdriver.Chrome('chromedriver',options=options)


brands=[["Barefoot","barefoot"],["Carlo Rossi","CARLO%20ROSSI"]]

for keyword in brands:
  try:

    url=f"https://shop.superfoodaruba.com/shop#!/?q={keyword[1]}&limit=48"

    driver.get(url)
    SCROLL_PAUSE_TIME = 5

    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info=driver.find_element(By.CSS_SELECTOR, 'div#products')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(True, {"class":["fp-item"]})
    productsList = []

    for product in productsTags:
      try:
        date = datetime.now().strftime("%d/%m/%Y")
        name = product.find("div",class_="fp-item-name notranslate").text.replace("\n","")
        price = product.find("span", class_="fp-item-base-price").text.strip().replace("ƒ","")
        link = "https://shop.superfoodaruba.com"+product.find("a",href=True)["href"]
        newProduct=[date,"Aruba","Superfood",keyword[0],name,float(price),link]
        productsList.append(newProduct)
      
      except Exception as err:
        print("ERROR "+keyword[0]+":"+name, err)

    for product in productsList:
      worksheet.append_row(product)

    data_df = pd.DataFrame(productsList)
    data_df.to_csv(f"{keyword[0]}_superfood.csv", index=False)
  except Exception as err:
    print("ERROR "+keyword[0]+":", err)


Proveedor - **Price Smart** (https://www.pricesmart.com/site/aw/en)
---


***> Marcas:***
*   *Barefoot*
* *La Marca*
* *Carlo Rossi*


In [5]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["La Marca","la+marca","marca"],["Barefoot","barefoot","barefoot"],["Carlo Rossi","carlo+rossi","carlo rossi"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.pricesmart.com/site/aw/en/search?_sq="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='div.container-elements')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("div", class_="search-product-box")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("p", class_="search-product-description").text.strip()
      price = product.find("span").text.replace("Afl.","").replace(",",".").strip()
      link = "https://www.pricesmart.com"+product.find(href=True)["href"]
      newProduct = [date,"Aruba","PriceSmart",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[2].lower())
      if marca > -1:
        productsList.append(newProduct)


    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_PriceSmart.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

Proveedor - **Ling and Sons** (https://shop.lingandsons.com/)
---


***> Marcas:***
*   *Barefoot*

In [6]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import numpy as np

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')# max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--incognito')

driver = webdriver.Chrome('chromedriver',options=options)


brands=[["Barefoot","barefoot"]]

for keyword in brands:
  try:  


    url = f"https://shop.lingandsons.com/search?q={keyword[1]}"
    driver.get(url)
    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info=driver.find_element(By.CSS_SELECTOR, 'div.grid--uniform')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(True, {"class":["product grid__item medium-up--one-third small--one-half slide-up-animation animated"]})
    productsList = []

    for product in productsTags:
                
                date = datetime.now().strftime("%d/%m/%Y")
                name = product.find("div",class_="product__title product__title--card text-center").text.replace("\n","")
                price = product.find("span", class_="product__price").text.strip().replace("\n","").replace("Afl","").replace("Regular price","")
                links = product.find(href=True)["href"]
                link = "https://shop.lingandsons.com"+links[0:links.find("?")]
                newProduct = [date,"Aruba","Ling and sons",keyword[0],name,float(price),link]
                productsList.append(newProduct)
    for product in productsList:
       worksheet.append_row(product) 

    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_Lingandsons.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)


# ANTIGUA Y BARBUDA

Proveedor - **Brydens** (https://brydensantigua.com/)
---

***> Marcas:***

* *La Marca*


In [7]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import numpy as np

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["La Marca","la+marca"]]

for keyword in brands:
  try:  
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')# max pantalla
    options.add_argument('--disable-extensions')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--incognito')

    driver = webdriver.Chrome('chromedriver',options=options)
    url = "https://brydensantigua.com/search?type=product&q="+keyword[1]
    driver.get(url)
    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info=driver.find_element(By.CSS_SELECTOR, 'div.products')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(True, {"class":["one-third"]})
    productsList = []

    for product in productsTags:
                date = datetime.now().strftime("%d/%m/%Y")
                name = product.find("span",class_="title").text
                price = product.find("span", class_="money").text.strip().replace("EC","").replace("$","")
                link = "https://brydensantigua.com"+product.find(href=True)["href"]
                newProduct = [date,"Antigua y Barbuda","Brydens antigua",keyword[0],name,float(price),link]
                productsList.append(newProduct)
    for product in productsList:
        worksheet.append_row(product) 

    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_brydensantigua.csv', index=False)
    print(data_df)
  except Exception as err:
    print("ERROR "+keyword[0]+":", err)


ERROR La Marca: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.products"}
  (Session info: headless chrome=105.0.5195.102)
Stacktrace:
#0 0x5570bff0b1a3 <unknown>
#1 0x5570bfcd7ac3 <unknown>
#2 0x5570bfd0f869 <unknown>
#3 0x5570bfd0fa41 <unknown>
#4 0x5570bfd436e7 <unknown>
#5 0x5570bfd2d10d <unknown>
#6 0x5570bfd413cc <unknown>
#7 0x5570bfd2d443 <unknown>
#8 0x5570bfd0348b <unknown>
#9 0x5570bfd04975 <unknown>
#10 0x5570bff872a0 <unknown>
#11 0x5570bff4281a <unknown>
#12 0x5570bff4234a <unknown>
#13 0x5570bff42e75 <unknown>
#14 0x5570bff8314b <unknown>
#15 0x5570bff431de <unknown>
#16 0x5570bff24c23 <unknown>
#17 0x5570bff4d6b8 <unknown>
#18 0x5570bff4d852 <unknown>
#19 0x5570bff66b60 <unknown>
#20 0x7fb229ec96db <unknown>



# BERMUDAS

Proveedor - **Wineonline** (https://www.wineonline.bm/)
---

***> Marcas:***
*   *Dark Horse*
*   *Orin Swift*
*   *Louis M Maritini*
* *Apothic*
* *La Marca*
*   *Phrophecy*

In [8]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


brands=[["Dark Horse", "dark horse"], ["Orin Swift","orin swift"], ["Apothic","apothic"], ["Prophecy","prophecy"],["Louis M Martini", "uis"], ["La Marca", "la marca"]]

for keyword in brands:

    try:
        options = webdriver.ChromeOptions()
        options.add_argument('--start-maximized')#max pantalla
        options.add_argument('--disable-extensions')
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')

        driver = webdriver.Chrome('chromedriver',options=options)
        driver.get("https://www.wineonline.bm/")

        WebDriverWait(driver, 10)\
            .until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.av_btn.av_go')))\
            .click()
            
        WebDriverWait(driver, 10)\
            .until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input#ctl00_txtVerboseSearch')))\
            .send_keys(keyword[1])
          
        WebDriverWait(driver, 10)\
            .until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input#ctl00_btnTextSearch')))\
            .click()

        info=driver.find_element(By.XPATH,"/html/body/form/div[3]/div[2]/div/div/div[1]/div/div/div[3]/div/div/table/tbody")
        soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
        productsTags = soup.findAll(class_="border_prods")
        print(len(productsTags))
        productsList = []

        for product in productsTags:
                date = datetime.now().strftime("%d/%m/%Y")
                name = product.find(class_="name name_padd equal-height_new_products_name").text.replace("\n", "")
                
                price = product.find("span", class_="productSpecialPrice").text.replace("$","")
             
                link = "https://www.wineonline.bm/"+product.find(href=True)["href"]
               
                newProduct = [date,"Bermudas","Wineonline",keyword[0],name,float(price),link]
                marca = name.lower().find(keyword[0].lower())
               
                if marca > -1:
                  productsList.append(newProduct)

        for product in productsList:
            worksheet.append_row(product) 

        data_df = pd.DataFrame(productsList)
        data_df.to_csv(keyword[0]+'_wineonline.csv', index=False)
        
    except Exception as err:
      print("ERROR "+keyword[0]+":", err)


6
6
3
3
2
2


Proveedor - **Marketplace** (https://www.marketplace.bm/)
---


***> Marcas:***
*   *Barefoot*
* *Apothic*


In [9]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import numpy as np

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["Apothic","apothic"],["Barefoot","barefoot"]]

for keyword in brands:
  try:  
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')# max pantalla
    options.add_argument('--disable-extensions')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--incognito')

    driver = webdriver.Chrome('chromedriver',options=options)
    url = f"https://www.marketplace.bm/shop#!/?q={keyword[1]}"
    driver.get(url)
    SCROLL_PAUSE_TIME = 5
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info=driver.find_element(By.CSS_SELECTOR, 'div#products')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(True, {"class":["fp-item-container"]})
    productsList = []
    for product in productsTags:
        try:
                date = datetime.now().strftime("%d/%m/%Y")
                name = product.find("div",class_="fp-item-name notranslate").text.replace("\n","")
                price = product.find("span", class_="fp-item-base-price").text.strip().replace("$","")
                link = "https://www.marketplace.bm"+product.find("a",href=True)["href"]
                newProduct = [date,"Bermudas","marketplace",keyword[0],name,float(price),link]
                productsList.append(newProduct)
        except Exception as err:
                print("ERROR "+keyword[0]+":"+name, err)

    for product in productsList:
        worksheet.append_row(product)
        
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_marketplace.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)


Proveedor - **Dropit** (https://www.dropit.bm/)
---

**Revisar links de cada producto-modal
***> Marcas:***

* *Apothic*


In [10]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import numpy as np

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


brands=[["Apothic","apothic"]]

for keyword in brands:
  try:  
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')# max pantalla
    options.add_argument('--disable-extensions')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--incognito')

    driver = webdriver.Chrome('chromedriver',options=options)
    url = f"https://www.dropit.bm/search/{keyword[1]}"
    driver.get(url)
    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info=driver.find_element(By.CSS_SELECTOR, 'div.items')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(True, {"class":["product"]})
    productsList = []
    for product in productsTags:
       try:
                date = datetime.now().strftime("%d/%m/%Y")
                name = product.find("div",class_="name").text
                prices = product.find("div", class_="sp-product-price")
                price = prices.findAll("span")[0].text.strip().replace("$","").replace("Sale price","")
                id = product.find("div",class_="image")["style"].replace('background-image:','').replace('url("https://storage.googleapis.com/sp-public/items/medium/','').replace('.jpg?v=','-').replace('");','').strip().rstrip('-')
                newId = id[0:id.find("-")]
                link = f"https://www.dropit.bm/search/apothic?catalogProduct={newId}"
                newProduct = [date,"Bermudas","dropit",keyword[0],name,float(price),link]
                productsList.append(newProduct)
       
       except Exception as err:
            print("ERROR "+keyword[0]+": "+name, err)

    for product in productsList:
        worksheet.append_row(product) 
        
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_dropit.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)


# BAHAMAS

Proveedor - **700 Wines and Spirits** (https://www.700winesandspirits.com)
---


***> Marcas:***
*   *Barefoot*
* *Apothic** (Sin registro en la página)
* *Carlo Rossi*


In [11]:
#Manual

Proveedor - **Flying Ductchman** (http://www.flyingdutchman.com/)
---


***> Marcas:***
*   *Barefoot*

In [12]:
# La página no existe

Proveedor - **Island Wine and Spirits** (http://islandwinesandspirits.net/)
---


***> Marcas:***

* *La Marca*


In [13]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import numpy as np

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["La Marca","la+marca"]]

for keyword in brands:
  
  try:  
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')# max pantalla
    options.add_argument('--disable-extensions')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--incognito')

    driver = webdriver.Chrome('chromedriver',options=options)
    url = f"http://islandwinesandspirits.net/?s={keyword[1]}"
    driver.get(url)
    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info=driver.find_element(By.CSS_SELECTOR, 'ul.list-posts')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(True, {"class":["post-infors-wrapper"]})
    productsList = []

    for product in productsTags:
        date = datetime.now().strftime("%d/%m/%Y")
        name = product.find("h2",class_="heading-title").text
        link = product.find("a",href=True)["href"]
        driver2 = webdriver.Chrome('chromedriver',options=options)
        driver2.get(link)
        info2=driver2.find_element(By.CSS_SELECTOR, 'p.price')
        soup2 = BeautifulSoup(info2.get_attribute('innerHTML'), "lxml")
        price = soup2.find("span",{"class":"woocommerce-Price-amount amount"}).get_text(strip=True).replace("$","")
        newProduct = [date,"Bahamas","islandwinesandspirits",keyword[0],name,float(price),link]
        productsList.append(newProduct)

    for product in productsList:
        worksheet.append_row(product) 
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_islandwinesandspirits.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)


Proveedor - **Oasis Bahamas** (https://oasisbahamas.com/)
---


***> Marcas:***

* *Apothic*
* *Carlo Rossi*



In [14]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import numpy as np

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


brands=[["Apothic","apothic"],["Carlo Rossi","carlo+rossi"]]

for keyword in brands:
  try:  
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')# max pantalla
    options.add_argument('--disable-extensions')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--incognito')

    driver = webdriver.Chrome('chromedriver',options=options)
    url = f"https://oasisbahamas.com/search?type=product&q={keyword[1]}"
    driver.get(url)
    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info=driver.find_element(By.CSS_SELECTOR, 'div.list-view-items')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(True, {"class":["grid-view-item"]})
    productsList = []
    for product in productsTags:
                date = datetime.now().strftime("%d/%m/%Y")
                name = product.find("div",class_="h4 grid-view-item__title").text
                price = product.find("span", class_="original is-bold qv-regularprice").text.strip().replace("$","")
                links = product.find("a",href=True)["href"]
                link = "https://oasisbahamas.com"+links[0:links.find("?")]
                newProduct = [date,"Bahamas","Oasis Bahamas",keyword[0],name,float(price),link]
                productsList.append(newProduct)

    for product in productsList:
        worksheet.append_row(product) 
        
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_oasisbahamas.csv', index=False)
  
  except Exception as err:
    print("ERROR "+keyword[0]+":", err)


# BRASIL

Proveedor - **Pão de Açúcar** (https://www.paodeacucar.com/)
---

***> Marcas:***
*   *Barefoot* 


In [15]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import numpy as np

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1




brands=[["Barefoot","barefoot&p=1"]]

for keyword in brands:
  try:  
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')# max pantalla
    options.add_argument('--disable-extensions')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--incognito')

    driver = webdriver.Chrome('chromedriver',options=options)
    url = f"https://www.paodeacucar.com/busca?terms={keyword[1]}"
    driver.get(url)
    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info=driver.find_element(By.CSS_SELECTOR, 'body')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")

    productsTags = soup.findAll(True, {"class":["MuiGrid-root gridstyles-sc-6scn59-0 dmpXNW MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-sm-6 MuiGrid-grid-md-4 MuiGrid-grid-lg-3 MuiGrid-grid-xl-3"]})
    productsList = []

    for product in productsTags:
      try:

        date=datetime.now().strftime("%d/%m/%Y")
        name = product.find("div",class_="product-cardstyles__Top-sc-1uwpde0-6 gDBBEX").text 
        price = product.find("div", class_="product-cardstyles__Price-sc-1uwpde0-11").text.strip().replace("R$","").replace(",","")
        link = "https://www.paodeacucar.com/"+product.find("a",href=True)["href"]
        newProduct= [date,"Brasil","Pão de Açúcar",keyword[0],name,float(price),link]
    
        productsList.append(newProduct)

      except Exception as err:
        print("ERROR:", err)
      
    for product in productsList:
      worksheet.append_row(product)

    data_df = pd.DataFrame(productsList)
    data_df.to_csv(f"{keyword[0]}_paodeacucar.csv",index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)


# COLOMBIA 

Proveedor - **Carulla** (https://www.carulla.com/)
---

***> Marcas:***
*   *Dark horse*
*   *Barefoot*
*   *Apothic*
* *Louis M Martini*
* *Carlo Rossi*

In [16]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np
import time

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
 
gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1
 

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--incognito')
driver = webdriver.Chrome('chromedriver',options=options)

brands=[["Dark Horse","dark%20horse", "dark horse"],["Barefoot","barefoot", "barefoot"],["Apothic","apothic","apothic"],["Louis M Martini","louis%20martini","louis m"],["Carlo Rossi","carlo%20rossi","carlo rossi"]]

for keyword in brands:
  try:
    url = f"https://www.carulla.com/{keyword[1]}?map=ft"
    driver.get(url)

    SCROLL_PAUSE_TIME = 5

    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info=driver.find_element(By.CSS_SELECTOR, 'div#gallery-layout-container')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(True, {"class":["vtex-product-summary-2-x-container vtex-product-summary-2-x-container--productDefaultDescription vtex-product-summary-2-x-containerNormal vtex-product-summary-2-x-containerNormal--productDefaultDescription overflow-hidden br3 h-100 w-100 flex flex-column justify-between center tc"]})
    productsList = []

    for product in productsTags:
      try:
        date = datetime.now().strftime("%d/%m/%Y")
        name = product.find("h3").text
        marca = name.lower().find(keyword[2].lower())
        price = product.find("span", class_="exito-vtex-components-4-x-currencyContainer").text.strip().replace("$","").replace(".","")
        link = "https://www.carulla.com"+product.find("a", class_="vtex-product-summary-2-x-clearLink vtex-product-summary-2-x-clearLink--productDefaultDescription h-100 flex flex-column",href=True)["href"]
        newProduct=[date,"Colombia","Carulla",keyword[0],name,float(price),link]
        if marca >-1:
          productsList.append(newProduct)
          print(newProduct)

      except Exception as err:
          print("ERROR "+keyword[0]+" "+name+" :", err)

    for product in productsList:
      worksheet.append_row(product)

    data_df = pd.DataFrame(productsList)
    data_df.to_csv(f"{keyword[0]}_carulla.csv", index=False)
    
  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

['16/09/2022', 'Colombia', 'Carulla', 'Dark Horse', 'Vino tinto cabernet sauvignon DARK HORSE MARCA EXCLUSIVA 750 Mililitro ', 52400.0, 'https://www.carulla.com/vino-tinto-cabernet-sauvignon-dark-horse-marca-exclusiva-750-mililitro-773311/p']
['16/09/2022', 'Colombia', 'Carulla', 'Dark Horse', 'Vino pinot noir DARK HORSE MARCA EXCLUSIVA 750 Mililitro ', 52400.0, 'https://www.carulla.com/vino-pinot-noir-dark-horse-marca-exclusiva-750-mililitro-874336/p']
['16/09/2022', 'Colombia', 'Carulla', 'Dark Horse', 'Vino rosado DARK HORSE MARCA EXCLUSIVA 750 Mililitro ', 52900.0, 'https://www.carulla.com/vino-rosado-dark-horse-marca-exclusiva-750-mililitro-869456/p']
['16/09/2022', 'Colombia', 'Carulla', 'Dark Horse', 'Vino sauvignon blanco DARK HORSE MARCA EXCLUSIVA 750 Mililitro ', 52900.0, 'https://www.carulla.com/vino-sauvignon-blanco-dark-horse-marca-exclusiva-750-mililitro-790199/p']
['16/09/2022', 'Colombia', 'Carulla', 'Barefoot', 'Vino moscato BAREFOOT 750 ml ', 39900.0, 'https://www.car

Proveedor - **Éxito** (https://www.exito.com/)
---


***> Marcas:***
*   *Dark horse*
*   *Barefoot* 
*   *Apothic*
* *Louis M Martini*
* *Carlo Rossi*

In [17]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np
import time

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
 
gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')# max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--incognito')

driver = webdriver.Chrome('chromedriver',options=options)
brands=[["Dark Horse","dark%20horse", "dark horse"],["Barefoot","barefoot", "barefoot"],["Apothic","apothic","apothic"],["Louis M Martini","louis%20martini","louis m"],["Carlo Rossi","carlo%20rossi","carlo rossi"]]

for keyword in brands:
  try:
    url = f"https://www.exito.com/{keyword[1]}?_q={keyword[1]}&map=ft"

    driver.get(url)
    SCROLL_PAUSE_TIME = 5

    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    time.sleep(SCROLL_PAUSE_TIME)
    info=driver.find_element(By.ID, 'gallery-layout-container')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(True, {"class":["vtex-product-summary-2-x-container vtex-product-summary-2-x-container--productDefaultDescription vtex-product-summary-2-x-containerNormal vtex-product-summary-2-x-containerNormal--productDefaultDescription overflow-hidden br3 h-100 w-100 flex flex-column justify-between center tc"]})
    productsList = []
    print(len(productsTags))

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("h3").text
      print(name)
      marca = name.lower().find(keyword[2].lower())
      price = product.find("span", class_="exito-vtex-components-4-x-currencyContainer").text.strip().replace("$","").replace(".","")
      link = "https://www.exito.com"+product.find("a", class_="vtex-product-summary-2-x-clearLink vtex-product-summary-2-x-clearLink--productDefaultDescription h-100 flex flex-column",href=True)["href"]
      newProduct=[date,"Colombia","Éxito",keyword[0],name,float(price),link]
      if marca >-1:
        productsList.append(newProduct)
        print(newProduct)

    for product in productsList:
      worksheet.append_row(product)

    data_df = pd.DataFrame(productsList)
    data_df.to_csv(f"{keyword[0]}_éxito.csv", index=False)
    
  except Exception as err:
    print("ERROR "+keyword[0]+":", err)


4
Vino pinot noir DARK HORSE MARCA EXCLUSIVA 750 Mililitro 
['16/09/2022', 'Colombia', 'Éxito', 'Dark Horse', 'Vino pinot noir DARK HORSE MARCA EXCLUSIVA 750 Mililitro ', 49900.0, 'https://www.exito.com/vino-pinot-noir-dark-horse-marca-exclusiva-750-mililitro-874336/p']
Vino tinto cabernet sauvignon DARK HORSE MARCA EXCLUSIVA 750 Mililitro 
['16/09/2022', 'Colombia', 'Éxito', 'Dark Horse', 'Vino tinto cabernet sauvignon DARK HORSE MARCA EXCLUSIVA 750 Mililitro ', 49900.0, 'https://www.exito.com/vino-tinto-cabernet-sauvignon-dark-horse-marca-exclusiva-750-mililitro-773311/p']
Taza Cyberpunk 2077 Silverhand 
Vino rosado DARK HORSE MARCA EXCLUSIVA 750 Mililitro 
['16/09/2022', 'Colombia', 'Éxito', 'Dark Horse', 'Vino rosado DARK HORSE MARCA EXCLUSIVA 750 Mililitro ', 49900.0, 'https://www.exito.com/vino-rosado-dark-horse-marca-exclusiva-750-mililitro-869456/p']
8
Vino blanco pinot grigio BAREFOOT 750 ml 
['16/09/2022', 'Colombia', 'Éxito', 'Barefoot', 'Vino blanco pinot grigio BAREFOOT 75

Proveedor - **PriceSmart** (https://www.pricesmart.com/site/co/en)
---

***> Marcas:***

* *La Marca*

In [18]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["La Marca","la+marca","marca"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.pricesmart.com/site/co/en/search?_sq="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='div.container-elements')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("div", class_="search-product-box")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("p", class_="search-product-description").text.strip().replace( "\n","")
      price = product.find("span").text.replace(".","").replace("$","").replace(",",".").strip()
      link = "https://www.pricesmart.com"+product.find(href=True)["href"]
      newProduct = [date,"Colombia","PriceSmart",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[2].lower())
      if marca > -1:
        productsList.append(newProduct)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_PriceSmart.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

# COSTA RICA 

Proveedor - **Delika** (https://www.delika.cr/)
---

***> Marcas:***
*   *Dark horse*
*   *Barefoot* 
*   *Apothic*
* *Orin Swift*
* *Louis M Martini*
* *Carlos Rossi*
* *J Vineyards*
* *MacMurray*
* *Edna Valley*

In [19]:
import numpy as np
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

from datetime import datetime
import pandas as pd
import numpy as np

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
 
gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--incognito')
driver = webdriver.Chrome('chromedriver',options=options)

brands=[["Dark Horse","dark+horse"],["La Marca","la+marca"],["Apothic","apothic"],["J Vineyards","j+vineyards"],["Orin Swift","orin+swift"],["Louis M Martini", "louis+martini"],["MacMurray","mcmurray"],["Edna Valley","edna+valley"]]
for keyword in brands:
  try:
    url= f"https://www.delika.cr/?s={keyword[1]}&post_type=product&product_cat=0"
    driver.get(url)

    SCROLL_PAUSE_TIME = 2

    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info=driver.find_element(By.ID,"content")
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(True, {"class":["product-inner"]})
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("h3", class_="woocommerce-loop-product__title").text
      price = product.find("span", class_="price").text.strip().replace("₡","").replace(",","")
      link = product.find("a",class_="product-loop-title", href=True)["href"]
      newProduct=[date,"Costa Rica","Delika",keyword[0],name,float(price),link]
      productsList.append(newProduct)
    
    for product in productsList:
     worksheet.append_row(product)

    data_df = pd.DataFrame(productsList)
    data_df.to_csv(f"{keyword[0]}_delika.csv",index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

Proveedor - **Corner Shop App** (https://cornershopapp.com/es-cr/)
---

***> Marcas:***
*   *Barefoot*




In [20]:
# Manual

Proveedor - **Vino CR** (https://vino.cr/)
---

***> Marcas:***
*   *La Marca*
* *Apothic*
* *Louis M Martini*
* *Ghost Pines*
* *Mcmurray*

In [21]:
#Manual


Proveedor - **Automercado** (https://automercado.cr/)
---

***> Marcas:***

* *Apothic*
* *Edna Valley*

In [22]:
# manual

# REPÚBLICA DOMINICANA

Proveedor - **almacen.do** (https://almacen.do/)
---
***> Marcas:***
*   *Dark horse*
*   *Apothic* 

In [23]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime 

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
 
gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["Dark Horse","dark+horse"],["Apothic","apothic"]]

for keyword in brands:
  try:
      url = f"https://almacen.do/?s={keyword[1]}&post_type=product"
      url_data = requests.get(url)
      content = url_data.text
      soup = BeautifulSoup(content, "lxml")
      productsTags = soup.findAll(True, {"class":["product-inner product-item__inner"]})
      productsList = []

      for product in productsTags:
        date=datetime.now().strftime("%d/%m/%Y")
        name = product.find("h2", class_="woocommerce-loop-product__title").text  
        price = product.find("span", class_="woocommerce-Price-amount amount").text.strip().replace("RD$","").replace(",","")
        link = product.find("a",class_="woocommerce-LoopProduct-link woocommerce-loop-product__link", href=True)["href"]
        newProduct= [date,"República Dominicana","almacendo",keyword[0],name,float(price),link]
        marca = name.lower().find(keyword[0].lower())
        if marca > -1:
          productsList.append(newProduct)
      
      for product in productsList:
        worksheet.append_row(product)

      data_df = pd.DataFrame(productsList)
      data_df.to_csv(f"{keyword[0]}_almacendo.csv",index=False)

  except Exception as err:
    print("ERROR:", err)

Proveedor - **Super Mercado Nacional** (https://supermercadosnacional.com/)
---

***> Marcas:***
*   *Barefoot*
* *La Marca*
* *Apothic*
* *Carlo Rossi*
* *Dark Horse*
* *Ghost Pines*
* *Edna Valley*
* *Louis M Martini*
* *William Hill*
* *Macmurray*

In [24]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import numpy as np

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')# max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--incognito')

driver = webdriver.Chrome('chromedriver',options=options)


brands=[["Barefoot","barefoot","barefoot"],["La Marca","prosecco%la%marca","prosecco la marca"],["Apothic","apothic", "apothic"], ["Ghost Pines","ghost%20pines", "ghost pines"],["Edna Valley", "edna%20valley", "edna valley"],["William Hill", "william%20hill", "William Hill"],["Louis M Martini", "louis%20martini", "louis martini"],["MacMurray", "macmurray","Macmurray"],["Carlo Rossi", "carlo%rossi","Carlo Rossi"],["Dark Horse", "dark%20horse","Dark Horse"]]

for keyword in brands:
  try:

    url="https://supermercadosnacional.com/catalogsearch/result/?q="+keyword[1]

    driver.get(url)
    SCROLL_PAUSE_TIME = 5

    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info=driver.find_element(By.CSS_SELECTOR, 'div#instant-search-results-container')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("li", class_="ais-Hits-item")
    productsList = []

    for product in productsTags:
      try:
        date = datetime.now().strftime("%d/%m/%Y")
        name = product.find("div",class_="product-item-tile__name").text
        marca = name.upper().find(keyword[2].upper())
        price = product.find("span", class_="after_special").text.strip().replace("RD$","").replace(",","")
        link = product.find("a",href=True)["href"]
        if marca > 0:
          newProduct=[date,"República Dominicana","Super Mercados Nacional",keyword[0],name,float(price),link]
          productsList.append(newProduct)
    
      except Exception as err:
        print("ERROR "+keyword[0]+":"+name, err)
        
    for product in productsList:
      worksheet.append_row(product)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

Proveedor - **Green** (https://www.green.com.do/)
---

***> Marcas:***
*   *Barefoot*
* *La Marca*
* *Apothic*
* *Carlo Rossi*





In [25]:
from numpy.ma.core import prod
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Apothic","apothic","apothic"],["Carlo Rossi","carlo+rossi","carlo rossi"],["Barefoot","barefoot","barefoot"],["La Marca","la+marca","la marca"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.green.com.do/?s="+keyword[1]+"&post_type=product"
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='body')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("div", class_="product-grid-item")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("h3").text.strip()
      price = product.find("span",class_="price").text.replace(",","").replace("RD$","").strip()
      link = product.find("a")["href"]
      newProduct = [date,"República Dominicana","Green",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[2].lower())
      if marca > -1:
        productsList.append(newProduct)

    for product in productsList:
        worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_green.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

Proveedor - **Almacenes Hatuey** (https://almaceneshatuey.com)
---

***> Marcas:***

* *Apothic*





In [26]:
from numpy.ma.core import prod
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Apothic","oth","apothic"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://almaceneshatuey.com/?s="+keyword[1]+"&post_type=product&dgwt_wcas=1"
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='div.ast-woocommerce-container')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("li")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("h2").text.strip()
      price = product.find("span",class_="price").text.replace(",","").replace("RD$","").strip()
      link = product.find("a")["href"]
      newProduct = [date,"República Dominicana","Almacenes Hatuey",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[2].lower())
      if marca > -1:
        productsList.append(newProduct)

    for product in productsList:
     worksheet.append_row(product)
      

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

Proveedor - **Wineliveryrd** (http://www.wineliveryrd.com/busqueda)
---

***> Marcas:***

* *Louis M Martini*





In [27]:
from numpy.ma.core import prod
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Louis M Martini","louis","louis m"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "http://www.wineliveryrd.com/busqueda"
    driver.get(url)

    SCROLL_PAUSE_TIME = 3

    WebDriverWait(driver, 10)\
            .until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input.form-control')))\
            .send_keys(keyword[1])
          
    WebDriverWait(driver, 10)\
            .until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btn.btn-default.btn-search')))\
            .click()

    time.sleep(SCROLL_PAUSE_TIME)
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.XPATH, value='/html/body/div[3]/div')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("div", class_="product_box box-product")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("h6").text.strip()
      price = product.find("div",class_="price-box").text.replace(",","").replace("RD$","").strip()
      link = product.find("a")["href"]
      newProduct = [date,"República Dominicana","Wineliveryrd",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[2].lower())
      if marca > -1:
        productsList.append(newProduct)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_wineliveryrd.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

# ECUADOR

Proveedor - **EPB** (https://tienda.epbvinosylicores.com/)
---

***> Marcas:***
*   *Barefoot*



In [28]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Barefoot","barefoot","barefoot"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://tienda.epbvinosylicores.com/busqueda.php?texto="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='div#products-cont')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("div", class_="product-card")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("h2").text.strip()
      price = product.find("span",class_="price").text.replace("$","").strip()
      link = "https://tienda.epbvinosylicores.com/"+product.find("img")["onclick"].replace("document.location.href='","")
      newProduct = [date,"Ecuador","EPB",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[2].lower())
      if marca > -1:
        productsList.append(newProduct)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_EPB.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)



# GUATEMALA



Proveedor - **Vinoteca** (https://vinoteca.gt/)
---
***> Marcas:***
*   *Dark horse*
*   *Apothic* 


In [29]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime 

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
 
gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


brands=[["Dark Horse","dark+horse"],["Apothic","apothic"]]
for keyword in brands:
  try:
    url = f"https://vinoteca.gt/search?type=product&q={keyword}"
    url_data = requests.get(url)
    content = url_data.text
    soup = BeautifulSoup(content, "lxml")
    productsTags = soup.findAll(True, {"class":["product-thumbnail__info"]})
    productsList = []

    for product in productsTags:

      date=datetime.now().strftime("%d/%m/%Y")   
      name = product.find("a", class_="product-thumbnail__title").text.replace("\n","")
      if product.find("span", class_="money"):    
        price = product.find("span", class_="money").text.strip().replace("Q","")     
      links = product.find("a", class_="product-thumbnail__title",href=True)["href"]
      link = "https://vinoteca.gt"+links[0:links.find("?")]
      newProduct= [date,"Guatemala","vinoteca",keyword[0],name,float(price),link]
      productsList.append(newProduct)


    for product in productsList:
        worksheet.append_row(product)

    data_df = pd.DataFrame(productsList)
    data_df.to_csv(f"{keyword[0]}_vinoteca.csv",index=False)
    
  except Exception as err:
    print("ERROR "+keyword[0]+":", err)


Proveedor - **Kemik** (https://www.kemik.gt/)
---

***> Marcas:***
*   *Barefoot*
* *Apothic*




In [30]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Barefoot","barefoot","barefoot"],["Apothic","apothic","apothic"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.kemik.gt/?s="+keyword[1]+"&post_type=product"
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='div.ais-hits')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("div",class_="ais-hits--item")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("h2").text.strip()
      price = product.find("p",class_="alg-hit__priceholder").text.replace(",","").replace("Q","").strip()
      link = product.find("a")["href"]
      newProduct = [date,"Guatemala","Kemik",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[2].lower())
      if marca > -1:
        productsList.append(newProduct)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_kemik.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

Proveedor - **Pacifiko** (https://www.pacifiko.com/)
---

***> Marcas:***
*   *Barefoot*


In [31]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Barefoot","barefoot","barefoot"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.pacifiko.com/index.php?category_id=0&search="+keyword[1]+"&submit_search=&route=product%2Fsearch"
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='div.products-list.row.nopadding-xs.so-filter-gird')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("div",class_="product-item-container")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("h4").text.strip()
      price = product.find("div",class_="price").text.replace(",","").replace("Q","").strip()
      link = "https://www.pacifiko.com"+product.find("a")["href"]
      newProduct = [date,"Guatemala","Pacifiko",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[2].lower())
      if marca > -1:
        productsList.append(newProduct)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_MeatShop.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

ERROR Barefoot: could not convert string to float: '18.00 \n20.00'


Proveedor - **MeatShop** (https://meatshop.gt/)
---

***> Marcas:***
*   *Barefoot*


In [32]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Barefoot","barefoot"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://meatshop.gt/index.php?route=product/search&search="+keyword[1]+"&description=true"
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='div.row.main-products.product-grid')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("div",class_="product-thumb product-wrapper")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("h4").text.strip()
      price = product.find("p",class_="price").text.replace(",","").replace("Q","").strip()
      link = product.find("a")["href"]
      newProduct = [date,"Guatemala","MeatShop",keyword[0],name,float(price),link]
      productsList.append(newProduct)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_MeatShop.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

ERROR Barefoot: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.row.main-products.product-grid"}
  (Session info: headless chrome=105.0.5195.102)
Stacktrace:
#0 0x55879310b1a3 <unknown>
#1 0x558792ed7ac3 <unknown>
#2 0x558792f0f869 <unknown>
#3 0x558792f0fa41 <unknown>
#4 0x558792f436e7 <unknown>
#5 0x558792f2d10d <unknown>
#6 0x558792f413cc <unknown>
#7 0x558792f2d443 <unknown>
#8 0x558792f0348b <unknown>
#9 0x558792f04975 <unknown>
#10 0x5587931872a0 <unknown>
#11 0x55879314281a <unknown>
#12 0x55879314234a <unknown>
#13 0x558793142e75 <unknown>
#14 0x55879318314b <unknown>
#15 0x5587931431de <unknown>
#16 0x558793124c23 <unknown>
#17 0x55879314d6b8 <unknown>
#18 0x55879314d852 <unknown>
#19 0x558793166b60 <unknown>
#20 0x7fcff8dcd6db <unknown>



Proveedor - **Eurovinos** (https://eurovinos.com.gt/)
---

***> Marcas:***
*   *Louis M Martini*
* *Abstract*
*Buscar por bodega Gallo y Louis M*


In [33]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Louis M Martini","/louis_m_m/"],["Orin Swift","/ej_gallo_winery/"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://eurovinos.com.gt/web/product-category/bodega"+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='ul.products')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("li")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("h2").text.strip()
      price = product.find("span",class_="woocommerce-Price-amount amount").text.replace(",","").replace("Q","").strip()
      link = product.find("a")["href"]
      newProduct = [date,"Guatemala","Eurovinos",keyword[0],name,float(price),link]
      productsList.append(newProduct)
      print(productsList)
    ''' for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_EPB.csv', index=False) '''

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

ERROR Louis M Martini: Message: no such element: Unable to locate element: {"method":"css selector","selector":"ul.products"}
  (Session info: headless chrome=105.0.5195.102)
Stacktrace:
#0 0x56529810b1a3 <unknown>
#1 0x565297ed7ac3 <unknown>
#2 0x565297f0f869 <unknown>
#3 0x565297f0fa41 <unknown>
#4 0x565297f436e7 <unknown>
#5 0x565297f2d10d <unknown>
#6 0x565297f413cc <unknown>
#7 0x565297f2d443 <unknown>
#8 0x565297f0348b <unknown>
#9 0x565297f04975 <unknown>
#10 0x5652981872a0 <unknown>
#11 0x56529814281a <unknown>
#12 0x56529814234a <unknown>
#13 0x565298142e75 <unknown>
#14 0x56529818314b <unknown>
#15 0x5652981431de <unknown>
#16 0x565298124c23 <unknown>
#17 0x56529814d6b8 <unknown>
#18 0x56529814d852 <unknown>
#19 0x565298166b60 <unknown>
#20 0x7f0f081686db <unknown>

ERROR Orin Swift: Message: no such element: Unable to locate element: {"method":"css selector","selector":"ul.products"}
  (Session info: headless chrome=105.0.5195.102)
Stacktrace:
#0 0x56529810b1a3 <unknown>
#1 

# HONDURAS

Proveedor - **Istmania** (https://market.istmania.hn/)
---

***> Marcas:***
* *Dark Horse*
* *Barefoot*
* *La Marca*
* *William Hill*
* *Louis M Martini*
* *Carlo Rossi*
* *Ghost Pines*
* *Mcmurray*
* *Apothic*


In [34]:
#MANUAL

Proveedor - **Tinto** (https://www.tintohn.com//)
---

***> Marcas:***
*   *La Marca*
* *Louis M Martini*
* *Ghost Pines*

In [35]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["La Marca","la+marca","marca"],["Louis M Martini","louis+martini","louis m"],["Ghost Pines","Ghost+Pines","Ghost Pines"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.tintohn.com/?s="+keyword[1]+"&post_type=product&type_aws=true"
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='ul.products.columns-3.tablet-columns-2.mobile-columns-2')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("li")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("h2").text.strip()
      price = product.find("span", class_="woocommerce-Price-amount amount").text.replace("L","").replace(",","").strip()
      link = product.find("a")["href"]
      newProduct = [date,"Honduras","Tinto",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[2].lower())
      if marca > -1:
        productsList.append(newProduct)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_tinto.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

Proveedor - **PriceSmart** (https://www.pricesmart.com/site/hn/en)
---

***> Marcas:***

* *La Marca*
* *Louis M Martini*
* *Ghost Pines*

In [36]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["La Marca","la+marca","marca"],["Louis M Martini","louis+martini","louis m"],["Ghost Pines","Ghost+Pines","Ghost Pines"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.pricesmart.com/site/hn/en/search?_sq="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='div.container-elements')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("div", class_="search-product-box")

    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("p", class_="search-product-description").text.strip().replace( "\n","")
      price = product.find("span").text.replace("L","").replace(",","").strip()
      link = "https://www.pricesmart.com"+product.find(href=True)["href"]
      newProduct = [date,"Honduras","PriceSmart",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[2].lower())

      if marca > -1:
        productsList.append(newProduct)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_PriceSmart.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

# ISLAS CAIMÁN

Proveedor - **Jacques** (https://www.jacquesscottonline.com/)
---

***> Marcas:***
*   *Dark Horse*
*   *Orin Swift*
* *J Vineyards*
* *Apothic*
* *Louis M Martini*

In [37]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime
import numpy as np

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["Barefoot","barefoot"],["Dark Horse","dark+horse"],["Orin Swift","orion+swift"],["J Vineyards","j+vineyards"],["Apothic","apothic"],["Louis M Martini","louis"]]


for keyword in brands:
    try:

        options = webdriver.ChromeOptions()
        options.add_argument('--start-maximized')#max pantalla
        options.add_argument('--disable-extensions')
        options.add_argument('--headless')
        options.add_argument('--incognito')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')

        driver = webdriver.Chrome('chromedriver',options=options)
        driver.get("https://www.jacquesscottonline.com/?s="+keyword[1]+"&post_type=product")


        WebDriverWait(driver, 10)\
            .until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.age-gate-submit-yes')))\
            .click()
          
        WebDriverWait(driver, 10)\
            .until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.shop-container')))\

        info=driver.find_element(By.XPATH,"/html/body/div[1]/main/div/div[1]/div/div[2]")
        soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")


        productsTags = soup.findAll(class_="box-text box-text-products")
        productsList = []

        for product in productsTags:
            date = datetime.now().strftime("%d/%m/%Y")
            name = product.find(class_="woocommerce-LoopProduct-link woocommerce-loop-product__link").text
            price = product.find("span", class_="woocommerce-Price-amount amount").text.replace("KYD $","")
            link = product.find(href=True)["href"]
            newProduct = [date,"Islas Caimán","Jacques Scott",keyword[0],name,float(price),link]
            marca = name.lower().find(keyword[0].lower())
            if marca > -1:
                  productsList.append(newProduct)

        for product in productsList:
            worksheet.append_row(product) 

        data_df = pd.DataFrame(productsList)
        data_df.to_csv(keyword[0]+'_jacques_cott.csv', index=False)
      
    except Exception as err:
          print("ERROR "+keyword[0]+":", err)

# ISLAS VIRGENES BRITÁNICAS (BRVI)

Proveedor - **First Choice Provisioning** (https://bvicompleteprovisioning.com/)(https://www.firstchoiceprovisioning.com/)
---
En cualquier link redirecciona al mismo sitio de busqueda 


***> Marcas:***
*   *Barefoot*
* *Apothic*


In [38]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["Apothic","apothic","apothic"],["Barefoot","barefoot","barefoot"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.firstchoiceprovisioning.com/index.php?route=product/search&search="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            
    info = driver.find_element(by=By.CSS_SELECTOR, value='div.product-layout.product-grid.col-lg-4.col-md-4.col-sm-4.col-xs-12')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("div",class_='caption')
    productsList = []

    for product in productsTags:
      try:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("div",class_="name").text.strip()
          price = product.find("p",class_="price").text.strip().replace(",","").replace("$","")
          link = product.find("a")["href"]
          newProduct = [date,"Islas Virgenes Britanicas","First Choice Provisioning",keyword[0],name,float(price),link]
          marca = name.lower().find(keyword[2].lower())
          if marca > -1:
            productsList.append(newProduct)

      except Exception as err:
          print("ERROR:", err)

    for product in productsList:
      worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_First_Choice_Provisioning.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

Proveedor - **RiteWay** (https://www.rtwbvi.com/)
---


***> Marcas:***

* *La Marca*
* *William Hill*
* *Ghost Pines*
* *Louis M Martini*


In [39]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["Ghost Pines","ghost+pines","ghost pines"],["La Marca","la+marca","la marca"],["William Hill","william+hill","william hill"],["Louis M Martini","louis+martini","louis m"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.riteway.vg/catalogsearch/result/?q="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            
    info = driver.find_element(by=By.CSS_SELECTOR, value='ol.products.list.items.product-items')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll('li')
    productsList = []

    for product in productsTags:
      try:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("a",class_="product-item-link").text.strip()+" "+ product.find("div",class_="pack-size__container").text.strip()
          price = product.find("span",class_="price").text.strip().replace(",","").replace("$","")
          link = product.find("a")["href"]
          newProduct = [date,"Islas Virgenes Britanicas","Rite Way",keyword[0],name,float(price),link]
          marca = name.lower().find(keyword[2].lower())
          if marca > -1:
            productsList.append(newProduct)
            print(newProduct)

      except Exception as err:
          print("ERROR:", err)

    for product in productsList:
      worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_rite_way.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

['16/09/2022', 'Islas Virgenes Britanicas', 'Rite Way', 'Ghost Pines', 'Ghost Pines Cabernet Sauvignon Wine 750ml', 26.99, 'https://www.riteway.vg/ghost-pines-n-s-cabernet-sauvignon-npcty-12-750ml.html']
['16/09/2022', 'Islas Virgenes Britanicas', 'Rite Way', 'La Marca', 'La Marca Prosecco 187ml', 6.49, 'https://www.riteway.vg/la-marca-procesecco-187ml.html']
['16/09/2022', 'Islas Virgenes Britanicas', 'Rite Way', 'La Marca', 'La Marca Prosecco 750ml', 21.99, 'https://www.riteway.vg/la-marca-prosecco-wine-750ml.html']
['16/09/2022', 'Islas Virgenes Britanicas', 'Rite Way', 'William Hill', 'William Hill Napa Valley Cabernet Sauvignon Wine 750ml', 54.99, 'https://www.riteway.vg/william-hill-est-np-vly-carbernet-sauvignon-750ml.html']
['16/09/2022', 'Islas Virgenes Britanicas', 'Rite Way', 'William Hill', 'William Hill Napa Valley Chardonnay Wine 750ml', 30.99, 'https://www.riteway.vg/william-hill-napa-valley-chardonnay.html']
['16/09/2022', 'Islas Virgenes Britanicas', 'Rite Way', 'Louis

# ISLAS VIRGENES DE ESTADOS UNIDOS (USVI)

Proveedor - **PriceSmart** (https://www.pricesmart.com/site/vi/en)
---

***> Marcas:***

* *La Marca*


In [40]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["La Marca","la+marca","marca"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.pricesmart.com/site/vi/en/search?_sq="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='div.container-elements')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("div", class_="search-product-box")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("p", class_="search-product-description").text.strip().replace( "\n","")
      price = product.find("span").text.replace( "$","")
      link = "https://www.pricesmart.com"+product.find(href=True)["href"]
      newProduct = [date,"Islas Virgenes de Estados Unidos","PriceSmart",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[2].lower())
      if marca > -1:
        productsList.append(newProduct)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_PriceSmart.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

Proveedor - **St John Gourmet** (https://st-john-gourmet-market.myshopify.com/)
---

***> Marcas:***

* *Apothic*


In [41]:
#Pagina en Mantenimiento

Proveedor - **Starfish Market** (https://starfishmarket.com/)
---

***> Marcas:***

* *Apothic*
* *Ghost Pines*
* *Louis M Martini*


In [42]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["Ghost Pines","pine","ghost pines"],["Apothic","thic","apothic"],["Louis M Martini","tini","louis m"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://starfishmarket.com/?s="+keyword[1]+"&post_type=product&dgwt_wcas=1"
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            
    info = driver.find_element(by=By.CSS_SELECTOR, value='ul.products.columns-5')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll('li')
    productsList = []

    for product in productsTags:
      try:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("h2").text.strip()
          price = product.find("span",class_="woocommerce-Price-amount amount").text.strip().replace(",","").replace("$","")
          link = product.find("a")["href"]
          newProduct = [date,"Islas Virgenes de Estados Unidos","Starfishmarket",keyword[0],name,float(price),link]
          marca = name.lower().find(keyword[2].lower())
          if marca > -1:
            productsList.append(newProduct)

      except Exception as err:
          print("ERROR:", err)

    for product in productsList:
      worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_starfishmarket.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

# ISLA DE SAN MARTIN

Proveedor - **EZShop** (https://ezshop.primesxm.com/)
---

***> Marcas:***

* *La Marca*
* *Apothic*

In [43]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["La Marca","la+marca","la marca"],["Apothic","apothic","apothic"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://ezshop.primesxm.com/catalogsearch/result/?q="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            
    info = driver.find_element(by=By.CSS_SELECTOR, value='ol.products.list.items.product-items')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll('li')
    productsList = []

    for product in productsTags:
      try:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("a",class_="product-item-link").text.strip().replace("\n","")
          price = product.find("span",class_="price").text.replace(",","").replace("$","")
          link = product.find("a")["href"]
          newProduct = [date,"Isla de San Martin","EZShop",keyword[0],name,float(price),link]
          marca = name.lower().find(keyword[2].lower())
          if marca > -1:
            productsList.append(newProduct)

      except Exception as err:
          print("ERROR:", err)

    for product in productsList:
      worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_EZShop.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

# JAMAICA

Proveedor - **Sampars** (https://shopsampars.com/)
---

***> Marcas:***

* *Carlo Rossi*

In [44]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["Carlo Rossi","carlo+rossi","carlo rossi"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://shopsampars.com/?s="+keyword[1]+"&post_type=product&dgwt_wcas=1"
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            
    info = driver.find_element(by=By.CSS_SELECTOR, value='main#content')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll('li')
    productsList = []

    for product in productsTags:
      try:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("h3").text
          price = product.find("span",class_="woocommerce-Price-amount amount").text.replace(",","").replace("U$","")
          link = product.find("a")["href"]
          newProduct = [date,"Jamaica","Shop Sampars",keyword[0],name,float(price),link]
          marca = name.lower().find(keyword[2].lower())
          if marca > -1:
            productsList.append(newProduct)

      except Exception as err:
          print("ERROR:", err)

    for product in productsList:
      worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_Shop_Sampars.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

Proveedor - **Loshusan Supermarket** (https://loshusansupermarket.com)
---

***> Marcas:***

* *Carlo Rossi*

In [45]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["Carlo Rossi","carlo+rossi","carlo rossi"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://loshusansupermarket.com/search.php?search_query="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            
    info = driver.find_element(by=By.CSS_SELECTOR, value='ul.ProductList.Clear')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll('li')
    productsList = []

    for product in productsTags:
      try:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("h1").text
          price = product.find("div",class_="ProductPriceRating").text.replace(",","").replace("$","").replace("JMD","")
          link = product.find("a")["href"]
          newProduct = [date,"Jamaica","Loshusan Supermarket",keyword[0],name,float(price),link]
          marca = name.lower().find(keyword[2].lower())
          if marca > -1:
            productsList.append(newProduct)

      except Exception as err:
          print("ERROR:", err)

    for product in productsList:
      worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_loshusan_supermarket.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

# MÉXICO

Proveedor - **Chedraui** (https://www.chedraui.com.mx/)
---

***> Marcas:***
* *Dark Horse*
* *Barefoot*
* *Apothic*
* *Carlo Rossi*
* *La Marca*
* *Louis M Martini*
* *Ghost Pines*
* *MacMurray*

In [46]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["Mac murray","mac%20murray","MacMurray"],["Dark Horse","dark%20horse","Dark Horse" ], ["Barefoot","barefoot","Barefoot"],["La Marca","la%20marca","La Marca"], ["Apothic","apothic","Apothic"],["Carlo Rossi","carlo%20rossi","Carlo Rossi"],["Louis M","louis%20m", "Louis M Martini"],["Ghost Pines","ghost%20pines","Ghost Pines"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

        url = "https://www.chedraui.com.mx/"+keyword[1]+"?_q="+keyword[1]+"&map=ft"
        driver.get(url)

        SCROLL_PAUSE_TIME = 5
        last_height = driver.execute_script("return document.body.scrollHeight")
            #Scroll para paginas cuyo renderizado depende de hacerlo
        while True:
                # Scroll down to bottom
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait to load page
                time.sleep(SCROLL_PAUSE_TIME)

                # Calculate new scroll height and compare with last scroll height
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

        info = driver.find_element(by=By.ID, value="gallery-layout-container")
        soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
        productsTags = soup.findAll("section")
        productsList = []

        for product in productsTags:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("h3").text
          price = product.find("span", class_="vtex-product-summary-2-x-currencyContainer").text.strip().replace(",","").replace("$","")
          link = "https://www.chedraui.com.mx"+product.find(href=True)["href"]
          newProduct = [date,"México","Chedraui",keyword[2],name,float(price),link]
          marca = name.lower().find(keyword[0].lower())
          if marca > -1:
              productsList.append(newProduct)
        
        for product in productsList:
            worksheet.append_row(product)

        data_df = pd.DataFrame(productsList)
        data_df.to_csv(keyword[0]+'_chedraui.csv', index=False)
        
  except Exception as err:
        print("ERROR "+keyword[0]+":", err)

Proveedor - **El Palacio de Hierro** (https://www.elpalaciodehierro.com/)
---

***> Marcas:***
* *Dark Horse*
* *Apothic*
* *La Marca*
* *Louis M Martini*
* *Ghost Pines*

In [47]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


brands=[["Dark Horse","dark%20horse","dark horse"],["La Marca","la%20marca","la marca"], ["Apothic","apothic","apothic"],["Louis M Martini","louis%20martini","louis m"],["Ghost Pines","ghost%20pines","ghost p"]]

for keyword in brands:

    try:
        url = "https://www.elpalaciodehierro.com/buscar?q="+keyword[1]
        url_data = requests.get(url)
        content = url_data.text
        soup = BeautifulSoup(content, "lxml")
        productsTags = soup.findAll(class_="l-plp-grid_item m-product")
        productsList = []

        for product in productsTags:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("a", class_="b-product_tile-name").text.replace("\n", "")
          price = product.find("span", class_="b-product_price-value").text.replace("\n", "").replace("$","").replace(",","")
          link = "https://www.elpalaciodehierro.com"+product.find(href=True)["href"]
          newProduct = [date,"México","El Palacio de Hierro",keyword[0],name,float(price),link]
          marca = product.find(class_="b-product_tile-brand").text.lower().find(keyword[2])
          if marca > -1:
              productsList.append(newProduct)

        for product in productsList:
            worksheet.append_row(product)

        data_df = pd.DataFrame(productsList)
        data_df.to_csv(keyword[0]+'_el_palacio_del_hierro.csv', index=False)
        
    except Exception as err:
        print("ERROR "+keyword[0]+":", err)

Proveedor - **Super Walmar - Superamat** (https://super.walmart.com.mx/)
---
***> Marcas:***
* *Dark Horse*
* *Barefoot*
* *Apothic*
* *Carlo Rossi*
* *Ghost Pines*
* *Louis M Martini*

In [48]:
#Manual

Proveedor - **Sams** (https://www.sams.com.mx/)
---
***> Marcas:***
* *Dark Horse*
* *Barefoot*
* *Mcmurray*


In [49]:
#Manual

Proveedor - **Bodegas Alianza** (https://www.bodegasalianza.com/)
---
***> Marcas:***
* *Barefoot*
* *Apothic*
* *Louis M Martini*
* *MacMurray*
* *Ghost Pines*
* *Carlo Rossi*



In [50]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Carlo Rossi","carlo%20rossi","Carlo Rossi"],["Ghost Pines","Ghost%20Pines","Ghost Pines"],["MacMurray","MacMurray","MacMurray"],["Apothic","apothic","apothic"],["Louis M Martini","Louis%20M%20Martini","louis m"],["Barefoot","barefoot","barefoot"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.bodegasalianza.com/"+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='div.product-list__wrapper')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(True, {"class":["product-item__wrapper"]})
    productsList = []

    for product in productsTags:
      try:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("h2").text.replace("\n","")
          price = product.find("span").text.replace( "$","").replace( ",","")
          link = product.find(href=True)["href"]
          newProduct = [date,"México","Bodegas Alianza",keyword[0],name,float(price),link]
          marca = name.lower().find(keyword[2].lower())
          if marca > -1:
            productsList.append(newProduct)

      except Exception as err:
          print("ERROR:", err)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_Bodegas_Alianza.csv', index=False)

  except Exception as err:
        print("ERROR "+keyword[0]+":", err)

ERROR: could not convert string to float: 'AGOTADO'
ERROR: could not convert string to float: 'AGOTADO'


Proveedor - **Mercado de Vinos** (https://mercadodevinos.mx/)
---
***> Marcas:***
* *Barefoot*
* *Apothic*
* *Orin Swift*
* *Louis M Martini*
* *Ghost Pines*
**Mcmurray*
* *Mount Peak*

In [51]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Ghost Pines","Ghost+Pines","Ghost Pines"],["MacMurray","MacMurray","MacMurray"],["Apothic","apothic","apothic"],["Louis M Martini","Louis+Martini","louis m"],["Barefoot","barefoot","barefoot"],["Mount Peak","mount+peak","mount peak"],["Orin Swift","orin+swift","orin swift"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://mercadodevinos.mx/search?type=product%2Carticle%2Cpage&options%5Bprefix%5D=last&q="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='main#content')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(True, {"class":["block-inner"]})
    productsList = []

    for product in productsTags:
      try:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("div",class_="product-block__title").text
          prices = product.find("div",class_="product-price")
          price = prices.findAll("span")[0].text.replace( "$","").replace( ",","")
          links = product.find(href=True)["href"]
          link = "https://mercadodevinos.mx"+links[0:links.find("?")]
          newProduct = [date,"México","Mercado de Vinos",keyword[0],name,float(price),link]
          productsList.append(newProduct)

      except Exception as err:
          print("ERROR:"+keyword[0]+" - "+name, err)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_mercado_de_vinos.csv', index=False)

  except Exception as err:
        print("ERROR "+keyword[0]+":", err)

Proveedor - **Lacomer** (https://www.lacomer.com.mx/)
---
***> Marcas:***
* *Barefoot*
* *Carlo Rossi*

In [52]:
#Manual

Proveedor - **Soriana** (https://www.soriana.com/)
---
***> Marcas:***
* *Barefoot*
* *Apothic*
* *Ghost Pines*
* *Carlo Rossi*


In [53]:
#Manual

Proveedor - **Costco** (https://www.costco.com.mx/)
---

***> Marcas:***

* *La Marca*
* *Apothic*
* *Orin Swift*
* *Louis M Martini*
* *MacMurray*
* *Mount Peak*


In [54]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["Apothic","apothic","apothic"],["La Marca","la marca","la+marca"],["Orin Swift","orin swift","orin+swift"],["Louis M Martini","louis m","louis+martini"],["MacMurray","macmurray","macmurray"],["Mount Peak","peak","mount+peak"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.costco.com.mx/rest/v2/mexico/products/search?fields=FULL&query="+keyword[2]+"&pageSize=24&lang=es_MX&curr=MXN"
    driver.get(url)

    SCROLL_PAUSE_TIME = 5
    time.sleep(SCROLL_PAUSE_TIME)
    
    info = driver.find_element(by=By.CSS_SELECTOR, value='div#webkit-xml-viewer-source-xml')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("products")
    productsList = []

    for product in productsTags:
      try:

          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("name").text
          marca = name.lower().find(keyword[1].lower())
          if marca > -1:
            price = product.find("price").value.text
            link = "https://www.costco.com.mx"+product.find_all("url")[-1].get_text()
            newProduct = [date,"México","Costco",keyword[0],name,float(price),link]
            productsList.append(newProduct)

      except Exception as err:
          print("ERROR:", err)

    for product in productsList:
        worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_costco.csv', index=False)

  except Exception as err:
        print("ERROR "+keyword[0]+":", err)

Proveedor - **Vino Premier** (https://vinopremier.mx/)
---

***> Marcas:***

* *La Marca*
* *Louis M Martini*


In [55]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["La Marca","la+marca","la marca"],["Louis M Martini","louis+m+martini","louis m"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://vinopremier.mx/busqueda?controller=search&s="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='div#js-product-list')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll('article')
    productsList = []

    for product in productsTags:
      try:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("h2").text
          price = product.find("span",class_="price").text.replace(",","").replace("$","")
          link = product.find(href=True)["href"]
          newProduct = [date,"México","Vinopremier",keyword[0],name,float(price),link]
          marca = name.lower().find(keyword[2].lower())
          if marca > -1:
            productsList.append(newProduct)

      except Exception as err:
          print("ERROR:", err)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_vinopremier.csv', index=False)

  except Exception as err:
        print("ERROR "+keyword[0]+":", err)

Proveedor - **Liverpool** (https://www.liverpool.com.mx/)
---

***> Marcas:***

* *Apothic*


In [56]:
#Manual

Proveedor - **La Europea** (https://www.laeuropea.com.mx/)
---

***> Marcas:***

* *Apothic*
* *Carlo Rossi*



In [57]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["Apothic","apothic","apothic"],["Carlo Rossi","carlo+rossi","carlo rossi"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.laeuropea.com.mx/catalogsearch/result/?q="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            
    info = driver.find_element(by=By.CSS_SELECTOR, value='ol.products.list.items.product-items')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll('li')
    productsList = []

    for product in productsTags:
      try:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("h2").text
          price = product.find("span",class_="price").text.replace(",","").replace("$","")
          link = product.find(href=True)["href"]
          newProduct = [date,"México","La Europea",keyword[0],name,float(price),link]
          marca = name.lower().find(keyword[2].lower())
          if marca > -1:
            productsList.append(newProduct)

      except Exception as err:
          print("ERROR:", err)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_la_europea.csv', index=False)

  except Exception as err:
    print("ERROR:"+keyword[0], err)

ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' object has no attribute 'text'
ERROR: 'NoneType' ob

Proveedor - **Club del Gourmet** (https://clubdelgourmet.com.mx/)
---

***> Marcas:***

* *Ghost Pines*



In [58]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["Ghost Pines","ghost%20pines","ghost pines"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://clubdelgourmet.com.mx/pages/search-result-page?q="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            
    info = driver.find_element(by=By.CSS_SELECTOR, value='div.epica-search-result-page__product-list.epica-search-result-page__product-list-grid')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll('a', class_="epica-search-result-page__product-item epica-search-result-page__product-item-grid")
    productsList = []

    for product in productsTags:
      try:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("div", class_="epica-search-result-page__product-item-info-title").text
          price = product.find("div",class_="epica-search-result-page__product-item-info-price").text.replace(",","").replace("$","")
          link = "https://clubdelgourmet.com.mx"+product.get('href')
          newProduct = [date,"México","Club del Gourmet",keyword[0],name,float(price),link]
          marca = name.lower().find(keyword[2].lower())
          if marca > -1:
            productsList.append(newProduct)

      except Exception as err:
          print("ERROR:", err)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_Club_del_Gourmet.csv', index=False)

  except Exception as err:
        print("ERROR "+keyword[0]+":", err)

ERROR Ghost Pines: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.epica-search-result-page__product-list.epica-search-result-page__product-list-grid"}
  (Session info: headless chrome=105.0.5195.102)
Stacktrace:
#0 0x558b7670b1a3 <unknown>
#1 0x558b764d7ac3 <unknown>
#2 0x558b7650f869 <unknown>
#3 0x558b7650fa41 <unknown>
#4 0x558b765436e7 <unknown>
#5 0x558b7652d10d <unknown>
#6 0x558b765413cc <unknown>
#7 0x558b7652d443 <unknown>
#8 0x558b7650348b <unknown>
#9 0x558b76504975 <unknown>
#10 0x558b767872a0 <unknown>
#11 0x558b7674281a <unknown>
#12 0x558b7674234a <unknown>
#13 0x558b76742e75 <unknown>
#14 0x558b7678314b <unknown>
#15 0x558b767431de <unknown>
#16 0x558b76724c23 <unknown>
#17 0x558b7674d6b8 <unknown>
#18 0x558b7674d852 <unknown>
#19 0x558b76766b60 <unknown>
#20 0x7fa7618cc6db <unknown>



Proveedor - **Alcornoque** (https://alcornoque.mx/)
---

***> Marcas:***

* *Orin Swift*



In [59]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["Orin Swift","orin+swift","orin swift"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:

    try:

      url = "https://alcornoque.mx/search?type=product&options%5Bprefix%5D=last&q="+keyword[1]
      driver.get(url)

      SCROLL_PAUSE_TIME = 3
      last_height = driver.execute_script("return document.body.scrollHeight")
          #Scroll para paginas cuyo renderizado depende de hacerlo
      while True:
              # Scroll down to bottom
              driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

              # Wait to load page
              time.sleep(SCROLL_PAUSE_TIME)

              # Calculate new scroll height and compare with last scroll height
              new_height = driver.execute_script("return document.body.scrollHeight")
              if new_height == last_height:
                  break
              last_height = new_height
              
      info = driver.find_element(by=By.CSS_SELECTOR, value='div.product-list.product-list--collection')
      soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
      productsTags = soup.findAll('div',class_="product-item__info")
      productsList = []

      for product in productsTags:
            try:
                date = datetime.now().strftime("%d/%m/%Y")
                name = product.find("a",class_="product-item__vendor link").text+" "+product.find("a",class_="product-item__title text--strong link").text
                price = product.find("span",class_="price").text.replace(",","").replace("$","").replace("Precio de venta","")
                links = product.find("a",class_="product-item__title text--strong link")["href"]
                link = "https://alcornoque.mx"+links[0:links.find("?")]
                newProduct = [date,"México","Alcornoque",keyword[0],name,float(price),link]
                marca = name.lower().find(keyword[2].lower())
                if marca > -1:
                  productsList.append(newProduct)

            except Exception as err:
                print("ERROR:", err)

      data_df = pd.DataFrame(productsList)
      data_df.to_csv(keyword[0]+'_alcornoque.csv', index=False)

      for product in productsList:
            worksheet.append_row(product)

    except Exception as err:
        print("ERROR "+keyword[0]+":", err)

# NICARAGUA

Proveedor - **Wine Store** (https://winestore.online.com.ni/)
---

***> Marcas:***

* *Apothic*


In [60]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Apothic","apothic"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://winestore.online.com.ni/Search/Search?pageIndex=1&keywords="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='ul.products.list-unstyled.list')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(True, {"class":["list-content"]})
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("h3").text.replace("\n","")
      price = product.find("h5").text.replace("Precio: $","").replace( "$","")
      link = "https://winestore.online.com.ni"+product.find(href=True)["href"]
      newProduct = [date,"Nicaragua","Wine Store",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[0].lower())
      if marca > -1:
        productsList.append(newProduct)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_Wine_Store .csv', index=False)

  except Exception as err:
        print("ERROR "+keyword[0]+":", err)

Proveedor - **Tu Nicaragua** (https://tunicaragua.com/)
---

***> Marcas:***

* *Apothic*


In [61]:
#Manual

# PANAMÁ

Proveedor - **Ribasmith** (https://www.ribasmith.com/)
---

***> Marcas:***
* *Dark Horse*
* *Barefoot*
* *Apothic*
* *Carlo Rossi*
* *Louis M Martini*


In [62]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import numpy as np

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
#Open our new sheet and add some data. select sheet by number
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


brands=[["Louis M Martini",'louis%20martini&narrow=%5B%5B"Marca"%2C"LOUIS%20MARTINI"%5D%5D'],["Apothic",'apothic&narrow=%5B%5B"Marca"%2C"APOTHIC"%5D%5D'],["Dark Horse","dark%20horse&narrow=%5B%5B%22Marca%22%2C%22DARK%20HORSE%22%5D%5D"],["Barefoot",'barefoot&narrow=%5B%5B"Marca"%2C"BAREFOOT"%5D%5D'],["Carlo Rossi",'carlos%20rossi&narrow=%5B%5B"Marca"%2C"CARLO%20ROSSI"%5D%5D']]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:

    url = "https://www.ribasmith.com/index.php/instantsearchplus/result/?q="+keyword[1]

    driver.get(url)

    WebDriverWait(driver, 10)\
        .until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'ul#isp_search_results_container')))\

    info=driver.find_element(By.ID,"isp_search_results_container")
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(class_="isp_grid_product")
    productsList = []
    
    for product in productsTags:
        date = datetime.now().strftime("%d/%m/%Y")
        name = product.find("img")["alt"]
        #Producto sin Oferta
        if product.find("span", class_="isp_product_price money"):
            price = product.find("span", class_="isp_product_price money").text
        #Producto con Oferta
        if product.find(class_="isp_product_price isp_compare_at_price_exist money"):
            price = product.find(class_="isp_product_price isp_compare_at_price_exist money").text.replace("$","")
        link = product.find(href=True)["href"].replace(" ", "%20")
        newProduct = [date,"Panamá","Ribasmith",keyword[0],name,float(price.replace("$","")),link]
        productsList.append(newProduct)

    for product in productsList:
        worksheet.append_row(product) 

    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_ribasmith.csv', index=False)

Proveedor - **La Bottega** (https://www.labottegapanama.com/)
---

***> Marcas:***
*   *Dark horse*
* *Carlo Rossi*
* *William Hill*
* *Louis M Martini*
* *MacMurray*
* *Mount Peack*
* *Orin Swift*
* *Apothic*
* *Barefoot*

In [63]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Barefoot","barefoot","barefoot"],["Apothic","apothic","apothic"],["Orin Swift","orin+swift","orin swift"],["Mount Peak","mount+peak","mount peak"],["MacMurray","mac+murray","murray"],["Louis M Martini","louis+martini","louis m"],["William Hill","william+hill","william hill"],["Carlo Rossi","carlo+rossi","carlo rossi"],["Dark Horse","dark+horse","dark horse"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.labottegapanama.com/catalogsearch/result/?q="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='ol.products.list.items.product-items')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("li")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("a", class_="product-item-link").text.strip()
      price = product.find("span", class_="price").text.replace( "B/.","").replace( "$","")
      link = product.find("a")["href"]
      newProduct = [date,"Panamá","La Bottega",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[2].lower())
      if marca > -1:
        productsList.append(newProduct)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_LaBottega.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

Proveedor - **PriceSmart** (https://www.pricesmart.com/site/pa/en/)
---

***> Marcas:***
*   *La Marca*

In [64]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["La Marca","la+marca","marca"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.pricesmart.com/site/pa/en/search?_sq="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='div.container-elements')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("div", class_="search-product-box")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("p", class_="search-product-description").text.strip().replace( "\n","")
      price = product.find("span").text.replace( "$","")
      link = "https://www.pricesmart.com"+product.find(href=True)["href"]
      newProduct = [date,"Panamá","PriceSmart",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[2].lower())
      
      if marca > -1:
        productsList.append(newProduct)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_PriceSmart.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

Proveedor - **Felipe Motta** (https://www.felipemotta.com/)
---

***> Marcas:***
*   *La Marca*

In [65]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["La Marca","la+marca"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.felipemotta.com/url/shop?page=1&searchOn="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.ID, value='ProductsList')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("div", class_="single-product")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("h4", class_="product-name").text.replace( "\n","")
      price = product.find("p", class_="normal-price").text.replace( "$","")
      link = "https://www.felipemotta.com"+product.find(href=True)["href"]
      newProduct = [date,"Panamá","Felipe Motta",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[0].lower())
      if marca > -1:
        productsList.append(newProduct)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_Felipe_Mottaa.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

# PERÚ

Proveedor - **Wong** (https://www.wong.pe/)
---

***> Marcas:***
*   *Dark horse*


In [66]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


brands=[["Dark Horse","dark%20horse"]]

for keyword in brands:

    try:
        url = "https://www.wong.pe/busca/?ft="+keyword[1]
        url_data = requests.get(url)
        content = url_data.text
        soup = BeautifulSoup(content, "lxml")
        productsTags = soup.findAll(True, {"class":["product-item"]})
        productsList = []

        for product in productsTags:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("a", class_="product-item__name").text
          price = product.find("span", class_="product-prices__value--best-price").text.strip().replace( "S/.","")
          link = product.find(href=True)["href"]
          newProduct = [date,"Perú","Wong",keyword[0],name,float(price),link]
          productsList.append(newProduct)
        
        for product in productsList:
          worksheet.append_row(product) 
        
        data_df = pd.DataFrame(productsList)
        data_df.to_csv(keyword[0]+'_wong.csv', index=False)
        
    except Exception as err:
        print("ERROR "+keyword[0]+":", err)

Proveedor - **Dcmall** (https://www.dcmall.pe/)
---

***> Marcas:***
*   *Dark horse*

**Pagina Web No Existe**

In [67]:
#No Exuste

Proveedor - **Lima Gourmet** (https://www.limagourmet.com/)
---

***> Marcas:***
*   *Dark horse*

**Pagina Web En Mantenimiento**

In [68]:
#En mantenimiento

Proveedor - **Vino Selección** (https://vinoseleccion.com.pe/)
---

***> Marcas:***
*   *Dark horse*

In [69]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


brands=[["Dark Horse","https://vinoseleccion.com.pe/dark-horse-california-x4/"]]

for keyword in brands:

    try:
        url = keyword[1]
        url_data = requests.get(url)
        content = url_data.text
        soup = BeautifulSoup(content, "lxml")
        productsTags = soup.findAll(True, {"class":["summary-container"]})
        productsList = []

        for product in productsTags:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("h1").text
          price = product.find("span", class_="woocommerce-Price-amount amount").text.strip().replace("S/","").replace(",",".")
          link = keyword[1]
          newProduct = [date,"Perú","Vino Selección",keyword[0],name,float(price),link]
          productsList.append(newProduct)
          
        for product in productsList:
          worksheet.append_row(product) 
       
        data_df = pd.DataFrame(productsList)

        data_df.to_csv(keyword[0]+'_vino_seleccion.csv', index=False)
        
    except Exception as err:
        print("ERROR "+keyword[0]+":", err)

Proveedor - **Cava16** (https://www.cava16.com/)
---

***> Marcas:***
*   *Dark horse*
* *Louis M Martini*
* *William Hill*
* *Ghost Pines*
* *Orin Swift*
* *Barefoot*

In [70]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


brands=[
    "Louis M Martini",
    "Dark Horse",
    "William Hill",
    "Ghost Pines",
    "Orin Swift",
    "Barefoot"
    ]


for keyword in brands:

    try:
        options = webdriver.ChromeOptions()
        options.add_argument('--start-maximized')#max pantalla
        options.add_argument('--disable-extensions')
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')

        driver = webdriver.Chrome('chromedriver',options=options) 

        url = "https://cava16.com/product-category/vinos/?count=36&paged=&filter_pais=usa&query_type_pais=or"
        driver.get(url)

        SCROLL_PAUSE_TIME = 5
        last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        info = driver.find_element(by=By.CSS_SELECTOR, value='ul.products-container')
        soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
        productsTags = soup.findAll("li")
        productsList = []

        for product in productsTags:
          try:
            date = datetime.now().strftime("%d/%m/%Y")
            name = product.find("h3", class_="woocommerce-loop-product__title").text
            price = product.findChildren("span", class_="price")[0].text.replace("S/","")
            link = product.find(href=True)["href"]

            if name.strip(" ")[0] == keyword.strip(" ")[0]:
              newProduct = [date,"Perú","Cava16",keyword,name,float(price),link]
              productsList.append(newProduct)

          except Exception as err:
              print("ERROR products:"+name, err)
              
        for product in productsList:
          worksheet.append_row(product) 


    except Exception as err:
        print("ERROR "+keyword+":", err)



Proveedor - **Plaza Vea** (https://www.plazavea.com.pe/)
---

***> Marcas:***
* *Barefoot*

In [71]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Barefoot","barefoot"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.plazavea.com.pe/search/?_query="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='div#category-page')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("div",class_="Showcase--food")
    productsList = []

    for product in productsTags:
      try:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("div", class_="Showcase__content")["title"]
          price = product.find("div", class_="Showcase__salePrice")["data-price"]
          link = product.find("a", class_="Showcase__name",href=True)["href"]
          newProduct = [date,"Perú","Plaza Vea",keyword[0],name,float(price),link]
          
          productsList.append(newProduct)
      
      except Exception as err:
        print("ERROR:", err)

    for product in productsList:
     worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_plazaVea.csv', index=False)

  except Exception as err:
        print("ERROR "+keyword[0]+":", err)

# PUERTO RICO

Proveedor - **Bodegarpr & BodegarGuaynabo** (https://bodegarpr.com/)
---

***> Marcas:***
*   *Dark horse*
*   *La Marca*
*   *Louis M Martini*
*   *Ghost Pines*
*   *J Vineyards*
*   *Macmurray*

In [72]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Dark Horse","dark-horse"],["La Marca","lamarca"],["Louis M Martini","louis-martini"],["Ghost Pines","ghost-pines"],["J Vineyards","j-vineyards"],["MacMurray","macmurray"]]

for keyword in brands:
  try:
    url = "https://bodegarpr.com/search?q="+keyword[1]
    url_data = requests.get(url)
    content = url_data.text
    soup = BeautifulSoup(content, "lxml")
    productsTags = soup.findAll(True, {"class":["product-detail"]})
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("a", class_="grid-link__title").text
      price = product.find("div", class_="product_price").text.strip().replace("$","")
      links = product.find(href=True)["href"]
      link = "https://bodegarpr.com"+links[0:links.find("?")]
      newProduct = [date,"Puerto Rico","Bodegarpr",keyword[0],name,float(price),link]
      productsList.append(newProduct)

    for product in productsList:
      worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_bodegarpr.csv', index=False)

  except Exception as err:
        print("ERROR "+keyword[0]+":", err)

Proveedor - **Bodegarpr-PR** (https://bodegar-mayaguez.com/)
---

***> Marcas:***
*   *La Marca*


In [73]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

#["Marca",["URL concatenada de busqueda"]]

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


brands=[["La Marca","lamarca"]]

for keyword in brands:
  try:
    url = "https://bodegar-mayaguez.com/search?q="+keyword[1]
    url_data = requests.get(url)
    content = url_data.text
    soup = BeautifulSoup(content, "lxml")
    productsTags = soup.findAll(True, {"class":["product-detail"]})
    productsList = []

    for product in productsTags:
      date = date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("a", class_="grid-link__title").text
      price = product.find("div", class_="product_price").text.strip().replace("$","")
      links = product.find(href=True)["href"]
      link = "https://bodegarpr.com"+links[0:links.find("?")]
      newProduct = [date,"Puerto Rico","Bodegarpr PR",keyword[0],name,float(price),link]
      productsList.append(newProduct)
    
    for product in productsList:
      worksheet.append_row(product)
    
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_bodegarpr_PR.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

Proveedor - **SuperMax ONLINE** (https://www.supermaxonline.com/)
---

***> Marcas:***
*   *Apothic*
*   *Barefoot*
*   *La Marca*
* *Louis M Martini*

In [74]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Apothic","apothic","Apothic"], ["Barefoot","barefoot","Barefoot"],["La Marca","la+marca","La Marca"],["Louis M Martini","louis+m","Louis M"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.supermaxonline.com/search-results.html?search="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
    info = driver.find_element(by=By.CSS_SELECTOR, value='div#products')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(True, {"class":["product-space"]})
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("h3").text
      price = product.find("p", class_="text-center precio").text.strip().replace("$","").replace(",","")
      link = "https://www.supermaxonline.com"+product.find(href=True)["href"]
      newProduct = [date,"Puerto Rico","SuperMax",keyword[0],name,float(price),link]
      marca = name.lower().find(keyword[2].lower())
      if marca > -1:
        productsList.append(newProduct)

    for product in productsList:
      worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_supermax.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

# EL SALVADOR

Proveedor - **Mistervinos** (https://www.mistervinosv.com/)
---
***> Marcas:***
* *Barefoot*


In [75]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1


#["Marca",["URL concatenada de busqueda"]]

brands=[["Barefoot","barefoot"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://www.mistervinosv.com/?s="+keyword[1]+"&search_id=product&post_type=product"
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='div#primary')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll("li")
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("a")["aria-label"]
      price = product.find("span", class_="woocommerce-Price-amount amount").text.strip().replace("$","").replace('"',"")
      link = product.find(href=True)["href"]
      newProduct = [date,"El Salvador","Mistervinos",keyword[0],name,float(price),link]
      productsList.append(newProduct)

    for product in productsList:
      worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_mistervinos.csv', index=False)

  except Exception as err:
   print("ERROR "+keyword[0]+":", err)

Proveedor - **KIP** (https://www.kip.sv/)
---
***> Marcas:***
* *Apothic*


In [76]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1



#["Marca",["URL concatenada de busqueda"]]

brands=[["Apothic","apothic"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:
  try:

    url = "https://kip.sv/catalogsearch/result/?q="+keyword[1]
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
    while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    info = driver.find_element(by=By.CSS_SELECTOR, value='div.ais-InfiniteHits')
    soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
    productsTags = soup.findAll(True, {"class":["product-item-info type-simple"]})
    productsList = []

    for product in productsTags:
      date = datetime.now().strftime("%d/%m/%Y")
      name = product.find("a", class_="product-item-link").text
      price = product.find("div", class_="price-box").text.strip().replace("$","").replace("/ UN","")
      link = product.find(href=True)["href"]
      newProduct = [date,"El Salvador","KIP",keyword[0],name,float(price),link]
      productsList.append(newProduct)

    for product in productsList:
      worksheet.append_row(product)
      
    data_df = pd.DataFrame(productsList)
    data_df.to_csv(keyword[0]+'_kip.csv', index=False)

  except Exception as err:
    print("ERROR "+keyword[0]+":", err)

# TRINIDAD Y TOBAGO

Proveedor - **ShupHub** (https://shuphub.com/)
---
***> Marcas:***

* *Barefoot*
* *Apothic*


In [77]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U').sheet1

brands=[["Barefoot","barefoot"],["Apothic","apothic"]]

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')#max pantalla
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

for keyword in brands:

    try:
        url = "https://shuphub.com/catalogsearch/result/index/?cat=&q="+keyword[1]+"&product_list_limit=36"
       
        driver.get(url)

        SCROLL_PAUSE_TIME = 2
        last_height = driver.execute_script("return document.body.scrollHeight")
        #Scroll para paginas cuyo renderizado depende de hacerlo
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        info = driver.find_element(by=By.CSS_SELECTOR, value='ol.products.list.items.product-items')
        soup = BeautifulSoup(info.get_attribute('innerHTML'), "lxml")
        productsTags = soup.findAll(True, {"class":["item product product-item"]})
        productsList = []

        for product in productsTags:
          date = datetime.now().strftime("%d/%m/%Y")
          name = product.find("a", class_="product-item-link").text.replace("\n","").strip()
          price = product.find("span", class_="price").text.replace("TTD$","").replace(",","")
          link = product.find(href=True)["href"]
          newProduct = [date,"Trinidad y Tobago","ShupHub",keyword[0],name,float(price),link]
          productsList.append(newProduct)

        for product in productsList:
          worksheet.append_row(product) 

        data_df = pd.DataFrame(productsList)
        data_df.to_csv(keyword[0]+'_ShupHub.csv', index=False)

    except Exception as err:
        print("ERROR "+keyword[0]+":", err)

# Alerta


In [78]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
from datetime import datetime

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
sh = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U')
worksheet0 = sh.get_worksheet(0)
rowsFinal = worksheet0.get_all_values()
rowsInitial = 1 + rowsInitial
rowsFinal = len(rowsFinal)+1
print(rowsFinal, rowsInitial)
cantidad = (rowsFinal-rowsInitial)/250
print(cantidad)
cuenta = 0
ciclo = 0
while rowsInitial < rowsFinal:
  worksheet0.update(f'h{rowsInitial}',f'= SI(HOY()-A{rowsInitial}>31,"Falso","Verdadero")',raw=False)
  rowsInitial = rowsInitial+1
  cuenta = 1 + cuenta
  if cuenta == 250:
    time.sleep(65)
    cuenta = 0
    ciclo = ciclo + 1
    print(ciclo)


21970 21323
2.588
1
2


In [79]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
sh = gc.open_by_key('1FDoJaJp5Zx0U6830gqGIrs0J6LOcjgesrMUDrBjP51U')
worksheet = sh.get_worksheet(3)

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.

dataAll = pd.DataFrame.from_records(rows).rename(columns={0:'Fecha', 1:'Tipo de Producto', 2:'Link al Producto', 3:'País', 4:'Proveedor', 5:'Marca', 6:'Precio'})
data = dataAll.iloc[:,0:7]
dataNew = data[data['Tipo de Producto'] != "CONSTANTE"]
dataNew = dataNew[dataNew['Link al Producto'] != ""]

dataNewNueva = dataNew[dataNew['Tipo de Producto'] == "NUEVO PRODUCTO"]
source_code1 = ""

if dataNewNueva.shape[0]>0:
  dataNewNueva.to_html('nueva.html')
  fname = "nueva.html"
  html_file = open(fname, 'r', encoding='utf-8')
  source_code1 = '''<table class="dataframe" style="font-family:arial,helvetica,sans-serif;" role="presentation" cellpadding="0" cellspacing="0" width="100%"
    border="0">
    <tbody>
        <tr>
            <td style="overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;"
                align="left">

                <div style="color: #89713c; line-height: 140%; text-align: left; word-wrap: break-word;">
                    <p style="font-size: 14px; line-height: 140%; text-align: center;"><span
                            style="box-sizing: border-box; font-weight: bold; user-select: text; color: #555555; font-family: Arial, 'Helvetica Neue', Helvetica, sans-serif; font-size: 22px; line-height: 30.8px;"><span
                                style="box-sizing: border-box; user-select: text; line-height: 30.8px; font-size: 22px;">
                                Tabla de Productos Nuevos</span></span></p>
                </div>

            </td>
        </tr>
    </tbody>
</table><br>''' + html_file.read().replace("<thead>",'''<thead style="background: #948473;
  background: -moz-linear-gradient(top, #afa396 0%, #9e9081 66%, #948473 100%);
  background: -webkit-linear-gradient(top, #afa396 0%, #9e9081 66%, #948473 100%);
  background: linear-gradient(to bottom, #afa396 0%, #9e9081 66%, #948473 100%);overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;">''').replace('<table border="1" class="dataframe">','''<table style="border: 6px solid #948473;
  background-color: #FFE3C6;  width: 100%;  text-align: center;"> ''').replace('<th>','''<th style="  border: 1px solid #948473;
  padding: 4px 4px;  font-size: 13px;"> ''')

else:
  source_code1 = '''<table class="dataframe" style="font-family:arial,helvetica,sans-serif;" role="presentation" cellpadding="0" cellspacing="0" width="100%"
    border="0">
    <tbody>
        <tr>
            <td style="overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;"
                align="left">

                <div style="color: #89713c; line-height: 140%; text-align: left; word-wrap: break-word;">
                    <p style="font-size: 14px; line-height: 140%; text-align: center;"><span
                            style="box-sizing: border-box; font-weight: bold; user-select: text; color: #555555; font-family: Arial, 'Helvetica Neue', Helvetica, sans-serif; font-size: 22px; line-height: 30.8px;"><span
                                style="box-sizing: border-box; user-select: text; line-height: 30.8px; font-size: 22px;">
                                No hay Productos Nuevos</span></span></p>
                </div>

            </td>
        </tr>
    </tbody>
</table><br>''' 

dataNewIncremento = dataNew[dataNew['Tipo de Producto'] == "INCREMENTÓ"]
source_code2 = ""

if dataNewIncremento.shape[0]>0:
  dataNewIncremento.to_html('Incremento.html')
  fname = "Incremento.html"
  html_file = open(fname, 'r', encoding='utf-8')
  source_code2 = '''<table class="dataframe" style="font-family:arial,helvetica,sans-serif;" role="presentation" cellpadding="0" cellspacing="0" width="100%"
    border="0">
    <tbody>
        <tr>
            <td style="overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;"
                align="left">

                <div style="color: #89713c; line-height: 140%; text-align: left; word-wrap: break-word;">
                    <p style="font-size: 14px; line-height: 140%; text-align: center;"><span
                            style="box-sizing: border-box; font-weight: bold; user-select: text; color: #555555; font-family: Arial, 'Helvetica Neue', Helvetica, sans-serif; font-size: 22px; line-height: 30.8px;"><span
                                style="box-sizing: border-box; user-select: text; line-height: 30.8px; font-size: 22px;">
                                Tabla de Productos con Incremento de Precios</span></span></p>
                </div>

            </td>
        </tr>
    </tbody>
</table><br>''' +html_file.read().replace("<thead>",'''<thead style="background: #948473;
  background: -moz-linear-gradient(top, #afa396 0%, #9e9081 66%, #948473 100%);
  background: -webkit-linear-gradient(top, #afa396 0%, #9e9081 66%, #948473 100%);
  background: linear-gradient(to bottom, #afa396 0%, #9e9081 66%, #948473 100%);overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;">''').replace('<table border="1" class="dataframe">','''<table style="border: 6px solid #948473;
  background-color: #FFE3C6;  width: 100%;  text-align: center;"> ''').replace('<th>','''<th style="  border: 1px solid #948473;
  padding: 4px 4px;  font-size: 13px;"> ''')
else:
  source_code2 = '''<table class="dataframe" style="font-family:arial,helvetica,sans-serif;" role="presentation" cellpadding="0" cellspacing="0" width="100%"
    border="0">
    <tbody>
        <tr>
            <td style="overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;"
                align="left">

                <div style="color: #89713c; line-height: 140%; text-align: left; word-wrap: break-word;">
                    <p style="font-size: 14px; line-height: 140%; text-align: center;"><span
                            style="box-sizing: border-box; font-weight: bold; user-select: text; color: #555555; font-family: Arial, 'Helvetica Neue', Helvetica, sans-serif; font-size: 22px; line-height: 30.8px;"><span
                                style="box-sizing: border-box; user-select: text; line-height: 30.8px; font-size: 22px;">
                                No hay productos con incremento de precio</span></span></p>
                </div>

            </td>
        </tr>
    </tbody>
</table><br>''' 

dataNewBajo = dataNew[dataNew['Tipo de Producto'] == "DISMINUYÓ"]
source_code3 = ""

if dataNewBajo.shape[0]>0:
  dataNewBajo.to_html('Bajo_html.html')
  fname = "Bajo_html.html"
  html_file = open(fname, 'r', encoding='utf-8')
  source_code3 = '''<table class="dataframe" style="font-family:arial,helvetica,sans-serif;" role="presentation" cellpadding="0" cellspacing="0" width="100%"
    border="0">
    <tbody>
        <tr>
            <td style="overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;"
                align="left">

                <div style="color: #89713c; line-height: 140%; text-align: left; word-wrap: break-word;">
                    <p style="font-size: 14px; line-height: 140%; text-align: center;"><span
                            style="box-sizing: border-box; font-weight: bold; user-select: text; color: #555555; font-family: Arial, 'Helvetica Neue', Helvetica, sans-serif; font-size: 22px; line-height: 30.8px;"><span
                                style="box-sizing: border-box; user-select: text; line-height: 30.8px; font-size: 22px;">
                               Tabla de Productos en promoción</span></span></p>
                </div>

            </td>
        </tr>
    </tbody>
</table><br>''' +html_file.read().replace("<thead>",'''<thead style="background: #948473;
  background: -moz-linear-gradient(top, #afa396 0%, #9e9081 66%, #948473 100%);
  background: -webkit-linear-gradient(top, #afa396 0%, #9e9081 66%, #948473 100%);
  background: linear-gradient(to bottom, #afa396 0%, #9e9081 66%, #948473 100%);overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;">''').replace('<table border="1" class="dataframe">','''<table style="border: 6px solid #948473;
  background-color: #FFE3C6;  width: 100%;  text-align: center;"> ''').replace('<th>','''<th style="  border: 1px solid #948473;
  padding: 4px 4px;  font-size: 13px;"> ''')
else:
  source_code3 = '''<table class="dataframe"  style="font-family:arial,helvetica,sans-serif;" role="presentation" cellpadding="0" cellspacing="0" width="100%"
    border="0">
    <tbody>
        <tr>
            <td style="overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;"
                align="left">

                <div style="color: #89713c; line-height: 140%; text-align: left; word-wrap: break-word;">
                    <p style="font-size: 14px; line-height: 140%; text-align: center;"><span
                            style="box-sizing: border-box; font-weight: bold; user-select: text; color: #555555; font-family: Arial, 'Helvetica Neue', Helvetica, sans-serif; font-size: 22px; line-height: 30.8px;"><span
                                style="box-sizing: border-box; user-select: text; line-height: 30.8px; font-size: 22px;">
                                No hay productos en promoción</span></span></p>
                </div>

            </td>
        </tr>
    </tbody>
</table><br>''' 

dataNewSinStock = dataNew[dataNew['Tipo de Producto'] == "SIN STOCK"]
source_code4 = ""

if dataNewSinStock.shape[0]>0:
  dataNewSinStock.to_html('SinStock.html')
  fname = "SinStock.html"
  html_file = open(fname, 'r', encoding='utf-8')
  source_code4 = '''<table class="dataframe" style="font-family:arial,helvetica,sans-serif;" role="presentation" cellpadding="0" cellspacing="0" width="100%"
    border="0">
    <tbody>
        <tr>
            <td style="overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;"
                align="left">

                <div style="color: #89713c; line-height: 140%; text-align: left; word-wrap: break-word;">
                    <p style="font-size: 14px; line-height: 140%; text-align: center;"><span
                            style="box-sizing: border-box; font-weight: bold; user-select: text; color: #555555; font-family: Arial, 'Helvetica Neue', Helvetica, sans-serif; font-size: 22px; line-height: 30.8px;"><span
                                style="box-sizing: border-box; user-select: text; line-height: 30.8px; font-size: 22px;">
                                Tabla de productos sin Stock</span></span></p>
                </div>

            </td>
        </tr>
    </tbody>
</table><br>''' +html_file.read().replace("<thead>",'''<thead style="background: #948473;
  background: -moz-linear-gradient(top, #afa396 0%, #9e9081 66%, #948473 100%);
  background: -webkit-linear-gradient(top, #afa396 0%, #9e9081 66%, #948473 100%);
  background: linear-gradient(to bottom, #afa396 0%, #9e9081 66%, #948473 100%);overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;">''').replace('<table border="1" class="dataframe">','''<table style="border: 6px solid #948473;
  background-color: #FFE3C6;  width: 100%;  text-align: center;"> ''').replace('<th>','''<th style="  border: 1px solid #948473;
  padding: 4px 4px;  font-size: 13px;"> ''')
else:
  source_code4 = '''<table class="dataframe" style="font-family:arial,helvetica,sans-serif;" role="presentation" cellpadding="0" cellspacing="0" width="100%"
    border="0">
    <tbody>
        <tr>
            <td style="overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;"
                align="left">

                <div style="color: #89713c; line-height: 140%; text-align: left; word-wrap: break-word;">
                    <p style="font-size: 14px; line-height: 140%; text-align: center;"><span
                            style="box-sizing: border-box; font-weight: bold; user-select: text; color: #555555; font-family: Arial, 'Helvetica Neue', Helvetica, sans-serif; font-size: 22px; line-height: 30.8px;"><span
                                style="box-sizing: border-box; user-select: text; line-height: 30.8px; font-size: 22px;">
                                Todos los productos tienen stock</span></span></p>
                </div>

            </td>
        </tr>
    </tbody>
</table><br>''' 

source_code = source_code1 + "<br>" +source_code2 + "<br>" +source_code3 + "<br>" +source_code4 + "<br>" 


import smtplib, ssl

port = 465  # For SSL
smtp_server = "smtp.gmail.com"
sender_email = 'rubicascrapper@gmail.com'  # Enter your address
receiver_email = 'miguel.moreno@rubica.net'  # Enter receiver address
password = 'fpprvbbugpewrtyd'
date = datetime.now().strftime("%d/%m/%Y")

from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# on crée un e-mail
message = MIMEMultipart("alternative")
# on ajoute un sujet
message["Subject"] = "[Gallo Scrapper] Resultados "+date
# un émetteur
message["From"] = sender_email
# un destinataire
message["To"] = receiver_email

# on crée un texte et sa version HTML
texte = '''
Hola Buen dia, revisa el resumen de la info minada de Gallo
'''
html0 = '''
<html>
<body>

    <table style="font-family:arial,helvetica,sans-serif;" role="presentation" cellpadding="0" cellspacing="0" width="100%"
    border="0">
    <tbody>
        <tr>
            <td style="overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;"
                align="left">

                <table width="100%" cellpadding="0" cellspacing="0" border="0">
                    <tbody>
                        <tr>
                            <td style="padding-right: 0px;padding-left: 0px;" align="center">

                                <img align="center" border="0"
                                    src="https://drive.google.com/uc?export=download&amp;id=1ChDIEvT5ioOQZwAf2ufLagl5-ZfxLh6i"
                                    alt="" title=""
                                    style="outline: none;text-decoration: none;-ms-interpolation-mode: bicubic;clear: both;display: inline-block;border: none;height: auto;float: none;width: 100%;max-width: 428px;"
                                    width="428">

                            </td>
                        </tr>
                    </tbody>
                </table>

            </td>
        </tr>
    </tbody>
</table>
<table style="font-family:arial,helvetica,sans-serif;" role="presentation" cellpadding="0" cellspacing="0" width="100%"
    border="0">
    <tbody>
        <tr>
            <td style="overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;"
                align="left">

                <div style="color: #89713c; line-height: 140%; text-align: left; word-wrap: break-word;">
                    <p style="font-size: 14px; line-height: 140%; text-align: center;"><span
                            style="box-sizing: border-box; font-weight: bold; user-select: text; color: #555555; font-family: Arial, 'Helvetica Neue', Helvetica, sans-serif; font-size: 22px; line-height: 30.8px;"><span
                                style="box-sizing: border-box; user-select: text; line-height: 30.8px; font-size: 22px;">Resultados
                                del Seguimiento a Distribuidores</span></span></p>
                </div>

            </td>
        </tr>
    </tbody>
</table>
<table style="font-family:arial,helvetica,sans-serif;" role="presentation" cellpadding="0" cellspacing="0" width="100%"
    border="0">
    <tbody>
        <tr>
            <td style="overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;"
                align="left">

                <div style="line-height: 140%; text-align: left; word-wrap: break-word;">
                    <p style="text-align: center; font-size: 14px; line-height: 140%;"><span
                            style="color: #888888; font-family: Arial, 'Helvetica Neue', Helvetica, sans-serif; font-size: 14px; line-height: 19.6px;">A
                            continuación se encuentras las tablas resumen del día de hoy para la <strong>Marca
                                Gallo</strong></span></p>
                </div>

            </td>
        </tr>
    </tbody>

</table>
<br> '''
html1 = '''<div class="u-row-container" style="padding: 10px;background-color: #444444">
    <div class="u-row"
        style="Margin: 0 auto;min-width: 320px;max-width: 600px;overflow-wrap: break-word;word-wrap: break-word;word-break: break-word;background-color: transparent;">
        <div style="border-collapse: collapse;display: table;width: 100%;height: 100%;background-color: transparent;">

            <div class="u-col u-col-100"
                style="max-width: 320px;min-width: 600px;display: table-cell;vertical-align: top;">
                <div style="height: 100%;width: 100% ;">
                    <div
                        style="height: 100%; padding: 0px;border-top: 0px solid transparent;border-left: 0px solid transparent;border-right: 0px solid transparent;border-bottom: 0px solid transparent;">

                        <table style="font-family:arial,helvetica,sans-serif;" role="presentation" cellpadding="0"
                            cellspacing="0" width="100%" border="0">
                            <tbody>
                                <tr>
                                    <td style="overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;"
                                        align="left">

                                        <div style="line-height: 140%; text-align: left; word-wrap: break-word;">
                                            <p style="font-size: 14px; line-height: 140%; text-align: center;"><span
                                                    style="color: #999999; font-size: 14px; line-height: 19.6px;">Visualiza
                                                    el Tablero Aquí</span></p>
                                        </div>

                                    </td>
                                </tr>
                            </tbody>
                        </table>

                        <table style="font-family:arial,helvetica,sans-serif;" role="presentation" cellpadding="0"
                            cellspacing="0" width="100%" border="0">
                            <tbody>
                                <tr>
                                    <td style="overflow-wrap:break-word;word-break:break-word;padding:10px;font-family:arial,helvetica,sans-serif;"
                                        align="left">

                                        <div align="center">
                                            <a target="_blank"
                                                href="https://datastudio.google.com/reporting/a11645a5-e133-41dc-b94b-ff73164a10ef"
                                                style="box-sizing: border-box;display: inline-block;font-family:arial,helvetica,sans-serif;text-decoration: none;-webkit-text-size-adjust: none;text-align: center;color: #89713c; background-color: #353233; border-radius: 4px;-webkit-border-radius: 4px; -moz-border-radius: 4px; width:100%; max-width:100%; overflow-wrap: break-word; word-break: break-word; word-wrap:break-word; mso-border-alt: none;">
                                                <span style="display:block;padding:10px 20px;line-height:120%;">
                                                    <p style="font-size: 14px; line-height: 120%;"><span
                                                            style="font-size: 14px; line-height: 16.8px;">Data
                                                            Studio</span></p>
                                                </span>
                                            </a>
                                        </div>

                                    </td>
                                </tr>
                            </tbody>
                        </table>

                    </div>
                </div>
            </div>
        </div>
    </div>
</body>
</html>
'''
html = html0 + source_code + html1


# on crée deux éléments MIMEText 
texte_mime = MIMEText(texte, 'plain')
html_mime = MIMEText(html, 'html')

# on attache ces deux éléments 
message.attach(texte_mime)
message.attach(html_mime)


context = ssl.create_default_context()
with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message.as_string())

receiver_email = "sonia@rubica.net"

with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message.as_string())

receiver_email = "felipe.jimenez@rubica.net"

with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message.as_string())

